<a href="https://colab.research.google.com/github/flatlandLucaDando/Flatland/blob/flatland_v_3_deterministic/Flatland_Railways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISTRUZIONI

Bisogna eseguire le caselle in ordine, alcune caselle sono molto importanti per configurare l'esempio da utilizzare.

*   Structure rail definisce i binari (destri, sinistri, su e giù)
*   Example definisce l'esempio (l'infrastruttura) che si vuole considerare
*   Rewards and Penalities definisce i valori delle rewards
*   Training flag definisce il training che si vuole considerare
*   In Configuration si può variare volendo la configurazione
*   Simulation Values sono i valori della simulazione, come la lunghezza degli episodi, il numero di episodi etc etc

Tutte queste celle di codice possono essere modificate per variare i test e training che si vogliono eseguire

---

P.s. La prima cella restituisce un errore ma non ci sono problemi, il tutto funge lo stesso

---




# Introduction 

---

This is a Workbook to execute the Flatland-RealWorld 3.06 code on colab

First thing is important to import some libraries



In [ ]:
import sys

if 'google.colab' in sys.modules:
    # Clone GitHub repository
    !git clone --single-branch --branch flatland_v_3_deterministic https://github.com/flatlandLucaDando/Flatland.git
    # Copy files required to run the code
    !pip install cloudpickle
    !pip install flatland-rl

    # Restart Runtime
    import os
    os.kill(os.getpid(), 9)

Cloning into 'Flatland'...
remote: Enumerating objects: 3479, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 3479 (delta 8), reused 0 (delta 0), pack-reused 3464
Receiving objects: 100% (3479/3479), 19.42 MiB | 20.89 MiB/s, done.
Resolving deltas: 100% (1806/1806), done.
     |████████████████████████████████| 2.2 MB 3.5 MB/s 
  Using cached pytest_runner-5.3.1-py3-none-any.whl (7.1 kB)
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 231 kB 37.1 MB/s 
     |████████████████████████████████| 253 kB 4.6 MB/s 
     |████████████████████████████████| 1.6 MB 32.8 MB/s 
     |████████████████████████████████| 245 kB 31.3 MB/s 
     |████████████████████████████████| 1.0 MB 31.8 MB/s 
     |████████████████████████████████| 197 kB 37.4 MB/s 
     |████████████████████████████████| 119 kB 33.2 MB/s 
     |████████████████████████████████| 62 kB 713 kB/s 
     |███████████████████

In [1]:
import time
import numpy as np
import os
from datetime import datetime
import random
import sys
from argparse import ArgumentParser, Namespace
from pathlib import Path
from pprint import pprint

import psutil
from torch.utils.tensorboard import SummaryWriter
import torch
from typing import Callable, Tuple, Optional, Dict, List
from numpy.random.mtrand import RandomState

# Structure Rail
Contein the basilar structure of the rail, with the high velocity rails and the right, left, nord or sud rails

With i_flag you can select the different rails configurations

In [2]:
i = 3

if i == 1:
	# One rail, so no right or left rails  
	right_rails = [(0,0)]
	left_rails = [(0,0)]
	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

if i ==2:
	# Rails where the direction is right
	right_rails = []
	for i in range(8):
		right_rails.append((5,i+6))
	# Rails where the direction is left
	left_rails = []
	for i in range(8):
		left_rails.append((6,i+6))

	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

if i == 3:
	# One rail, so no right or left rails  
	right_rails = [(0,0)]
	left_rails = [(0,0)]
	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

# Example

In [3]:
from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transition_map import GridTransitionMap

'''
###############################################################
######################   EXAMPLE LUCA  #########################
###############################################################
'''

# Example generate a rail given a manual specification,
# a map of tuples (cell_type, rotation)

# Example 2 simple railway with 2 stations and two rails and multiple rails in the stations
railway_example = [[(0,0)] * 25,
				   [(0,0)] * 6 + [(9,270)] +[(1,90)]*13 + [(8,90)]  +[(0,0)] *4,
				   [(0,0)] * 6 +[(1,0)]+ [(9,270)] + [(1,90)]*10 + [(8,90)]  + [(0,0)]+ [(1,0)] +[(0,0)] *4,
				   [(0,0)] * 6 +[(1,0)]*2 +  [(0,0)] * 10+ [(1,0)] + [(0,0)]+ [(1,0)]  + [(0,0)] *4,
				   [(7, 270)] + [(1,90)] * 3 + [(8,90)] + [(0,0)] + [(1,0)]*2 +[(0,0)] *2 + [(8,0)] +  [(1,90)] * 3+ [(8,90)]+ [(0,0)] *3 + [(1,0)] +[(0,0)]+  [(1,0)] +[(0,0)]+ [(8,0)]+   [(1,90)] + [(7, 90)] ,
				   [(7, 270)] + [(1,90)] * 3 + [(5,270)] + [(2,270)] + [(5,0)] + [(2,90)] + [(10,90)] + [(2,270)]+ [(2,90)]  +[(1,90)] * 3+ [(10,270)] + [(10,90)]  + [(2,270)] + [(1,90)] + [(5,180)] + [(1,90)] + [(5,270)] + [(1,90)]+  [(5,180)] +[(1,90)]  + [(7, 90)] ,
				   [(7, 270)] + [(1,90)] * 3 + [(10,270)] + [(2,90)] +[(2,90)] +[(1,90)] + [(10,270)] + [(2,90)] + [(1,90)] *5+ [(10,270)] + [(2,90)]+ [(1,90)] + [(10,270)]+ [(1,90)] + [(10,270)]  +[(1,90)] + [(10,270)] +[(1,90)] + [(7, 90)],
				   [(0,0)] * 25,
				   [(0,0)] * 25]

# wheight and height of the grid
height = len(railway_example)
width = len(railway_example[0])

# creating the transition map
rail_env_transitions = RailEnvTransitions()
rail = GridTransitionMap(width=width, height=height, transitions=rail_env_transitions)

for r in range(height):
	for c in range(width):
		rail_spec_of_cell = railway_example[r][c]
		index_basic_type_of_cell_ = rail_spec_of_cell[0]
		rotation_cell_ = rail_spec_of_cell[1]
		if index_basic_type_of_cell_ < 0 or index_basic_type_of_cell_ >= len(rail_env_transitions.transitions):
			print("ERROR - invalid rail_spec_of_cell type=", index_basic_type_of_cell_)
		basic_type_of_cell_ = rail_env_transitions.transitions[index_basic_type_of_cell_]
		effective_transition_cell = rail_env_transitions.rotate_transition(basic_type_of_cell_, rotation_cell_)
		rail.set_transitions((r, c), effective_transition_cell)

# No high velocity lines, so make a (0,0) position
	av_line = (0,0)
	
	# TODO sistema i binari destra e sinistra e su e giù
	# Rails where the direction is right
	right_rails = [(0,0)]
	# Rails where the direction is left
	left_rails = [(0,0)]
	down_rails = [(0,0)]
	up_rails = [(0,0)]


# Data Structures

## Convoy


In [4]:
from enum import Enum
import itertools

# Type of a convoy, can be high velocity or regional
# The velocity are given by default depending on the type of convoy, 360 for HV, 180 for IC, 120 for Regional
class Type_of_convoy(Enum):
	HIGH_VELOCITY = 1
	INTERCITY = 2
	REGIONAL = 3

# A convoy is a locomotive + wagons.
class Convoy:

	# For the id
	id_iter = itertools.count()

	def __init__(self, train_type, schedule = []):

		# identifier of the train
		self.id = next(Convoy.id_iter)
		# type of train (High velocity, Intercity, regional)
		self.train_type = train_type
		# schedule of the train
		self.schedule = []

		if train_type == Type_of_convoy.HIGH_VELOCITY:
			self.maximum_velocity = 1
		if train_type == Type_of_convoy.INTERCITY:
			self.maximum_velocity = 1/2
		if train_type == Type_of_convoy.REGIONAL:
			self.maximum_velocity = 1/3

	def add_train_run(self, train_run):
		self.schedule.append(train_run)

	# Discover the starting time of a certain run
	def starting_time(self, run_number):
		return self.schedule[run_number][0]

	# Convert velocity (maximum possible velocity is 360)
	def velocity_conversion(self):
		print(self.maximum_velocity * 360)

	# Verificate that a schedule is possible (if someone want to write manually)
	def schedule_verification(self, schedule, num_trains_run):
		if type(schedule) == list:
			row_num = len(schedule)
			column_num = len(schedule[0])
			if row_num >  1:
				for num_of_runs in range(num_trains_run - 1):
					for steps in range(len(schedule) - 1):
						if schedule[num_of_runs][step + 1] <= schedule[num_of_runs][step]:
							print('==========================================================')
							print('The time to connect stations',step,'and',step + 1,'have to be > 0')

			else:
				for steps in range(len(schedule) - 1):
					if schedule[num_of_runs][step + 1] <= schedule[num_of_runs][step]:
						print('==========================================================')
						print('The time to connect stations',step,'and',step + 1,'have to be > 0')
		else:
			print('A schedule should comprend different stations')
		return True


## Line

In [5]:
import itertools

# A line is considered from a city to another, tipically joining several cities
class Line:

	# For the id
	id_iter = itertools.count()

	def __init__(self, type_line, stations, stops):
		# ID of the line
		self.id = next(Line.id_iter)
		# type of line (High velocity or regional)
		self.type_line = type_line
		# Stations where the line pass from
		self.stations = stations
		# Stops are stations where the train have to stop
		# Is an array with 0 where not stop and 1 where train stops
		self.stops = stops

		if type(stations) == int or type(stops) == int:
			print('The stations of a line should be more than one, and the dimension of the stops should be the same of the stations')
		else:
			if(len(stations)) != (len(stops)):
				print('Stations and Stops have to be the same lenght')


	def inversion_of_line(self):
		self.direction = self.direction * -1

## Rail Connection

In [6]:
from enum import Enum
import itertools

# A connection can be: High velocity or normal
# The velocity are given by default depending on the type of connection, 360 for HV and 120 for normal
class Connection_type(Enum):
	HIGH_VELOCITY_RAIL = 1
	NORMAL_RAIL = 2


# A physical connection between two stations
class Rail_connection:

	# For the id
	id_iter = itertools.count()

	def __init__(self, station_a, station_b, rail_connection_type, additional_runtime_percent, max_speed_usable: int = 1/2):
		# each railway section has an id
		self.id = next(Rail_connection.id_iter)
		# Station A and B are the two connected stations
		self.station_a = station_a
		self.station_b = station_b
		# A connection can be: High velocity or normal
		self.rail_connection_type = rail_connection_type
		# Maximum speed usable in the rails
		self.max_speed_usable = max_speed_usable
		# Additional Runtime Percent is the percent [0-1] of the min run time that is added to the min run time, if the train is on schedule.
		# In general, he actual run time is computed as min run time + max(0, (min run time*additionalRuntimePercent)-actual_delay).
		self.additional_runtime_percent = additional_runtime_percent

		if rail_connection_type == Connection_type.HIGH_VELOCITY_RAIL:
			self.max_speed_usable = 1
		if rail_connection_type == Connection_type.NORMAL_RAIL:
			self.max_speed_usable = 1 / 2

## Station

In [7]:
from flatland.core.grid.grid_utils import Vec2dOperations as Vec2d

# Station
class Station:

	# TODO metti valori precisi per min wait time (lis)
	def __init__(self, name, position, capacity, min_wait_time, additional_wait_percent, importance, railway_topology):
		# Name of the station (e.g. Milano, Torino etc etc)
		self.name = name
		# Position (y,x) of the station in the railway
		self.position = position
		# Capacity of the station, num of rails in the station
		self.capacity = capacity
		# Minimum wait time for the trains, a train can't stop less than the min wait time
		self.min_wait_time = min_wait_time
		# Additional wait percent is the percent [0-1] of the minWaitTime that is added to the minWait at each stop, if the train is on schedule. 
		# In general, the actual (runtime) stopping time is computed as minWaitTime + max(0, (minWaitTime*additionalWaitTimePercent)-actual_delay).
		self.additional_wait_percent = additional_wait_percent
		# Stations have different importance depending on how much they are big and how much people they transport depending on the time
		self.importance = importance
				# Rails of the station
		self.rails = self.calculate_rails(railway_topology)
		return


	def time_in_station(self, train_velocity):
		# The len of the rails is given by the station
		len_rails = len(self.rails_in_station[0])
		# The time needed is given by the formula (len * 1/velocity + waiting time + 10% of time)
		time_needed =  len_rails * int(pow(train_velocity, -1)) + self.min_wait_time[0]
		time_needed += int(time_needed/10)
		return time_needed

	def calculate_rails(self, railway_topology):
		# Number of rails of the station
		num_of_rails = self.capacity
		center_of_station = self.position 
		rail_shape = railway_topology.grid.shape

		#Flag
		left = True  # Flag to understand where to go right or left
		north = True    # Flag to understand where to go right or left

		# Indicating the incrementing number north or sud (in case of horizontal station), east ovest (in case of vertical stations)
		difference_from_original = 0 

		# Counter
		counter_of_rails = 1
		# Rails of the station, has the position of the rails. Each row is a rail
		self.rails = []
  
		# Counter to check the station is well positioned, to avoid the while goes for eternity
		counter = 0

		# Starting position the center of station
		current_position = (self.position[0], self.position[1])
  
		# The first rail coincide with the position of the station
		self.rails.append(current_position)

		while counter_of_rails < num_of_rails:

			counter += 1

			if counter > 500:
				raise ImportError('The position of the station, or the capacity should be different, check for the right position or capacity, cant calculate the rails')

			# Horizontal rail
			# is position inside the grid?
			elif current_position[0] >= rail_shape[0] or current_position[0] < 0 or current_position[1] >= rail_shape[1] or current_position[1] < 0:
				continue 
			
			# Starting going up to check the rails
			elif railway_topology.grid[current_position] == 1025 and north:
				# Going to up
				new_pos = (-1, 0)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)
				# If not 0 the rail is in the station
				if railway_topology.grid[current_position] != 0:
					self.rails.append(current_position)
					counter_of_rails += 1
					continue
				else:
					current_position = (self.position[0], self.position[1])
					north = False
					continue			

			elif railway_topology.grid[current_position] == 1025 and not north:
				# Going to up
				new_pos = (1, 0)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)
				# If not 0 the rail is in the station
				if railway_topology.grid[current_position] != 0:
					self.rails.append(current_position)
					counter_of_rails += 1
					continue
				else:
					continue
 
			# Vertical rail same as for horizontal, but starting from south and north, and then left and right
			elif railway_topology.grid[current_position] == 32800 and left:
				# Going to up
				new_pos = (0, -1)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)
				# If not 0 the rail is in the station
				if railway_topology.grid[current_position] != 0:
					self.rails.append(current_position)
					counter_of_rails += 1
					continue
				else:
					current_position = (self.position[0], self.position[1])
					left = False
					continue			

			elif railway_topology.grid[current_position] == 32800 and not left:
				# Going to up
				new_pos = (0, 1)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)
				# If not 0 the rail is in the station
				if railway_topology.grid[current_position] != 0:
					self.rails.append(current_position)
					counter_of_rails += 1
					continue
				else:
					continue

		# Eliminate duplicates
		rails_position = []
		rails_position_single = []
		for i in range(len(self.rails)):
			for j in range(len(self.rails[i])):
				if self.rails[i][j] not in rails_position_single:
					rails_position_single.append(self.rails[i][j])
			rails_position.append(rails_position_single)
			rails_position_single = []
		self.rails = rails_position

## Train Run

In [8]:
# A train run is the run on a line for a train (e.g. Genova-Milano --- Milano-Genova)
# The train run consider each intermediate station the train has to pass between the two "principal" stations
class Train_run:

	def __init__(self, line_belongin, starting_time, from_depot: bool = False, to_depot: bool = False, inverse_train_direction: bool = False):
		# Line in which the train run is, so it contein informations about the stations to stop and etc etc
		self.line_belongin = line_belongin
		# Starting time of the run
		self.starting_time = starting_time
		# FromDepot indicates whether this run starts from a depot. Similar for ToDepot.
		self.from_depot = from_depot
		self.to_depot = to_depot
		# If inverse line direction 
		self.inverse_train_direction = inverse_train_direction



# Utils function 

In [9]:
from enum import IntEnum
from typing import NamedTuple
from flatland.core.grid.grid4 import Grid4TransitionsEnum

class RailEnvActions(IntEnum):
    DO_NOTHING = 0  # implies change of direction in a dead-end!
    MOVE_LEFT = 1
    MOVE_FORWARD = 2
    MOVE_RIGHT = 3
    STOP_MOVING = 4
    REVERSE = 5

    @staticmethod
    def to_char(a: int):
        return {
            0: 'B',
            1: 'L',
            2: 'F',
            3: 'R',
            4: 'S',
            5: 'I',
        }[a]

    @classmethod
    def is_action_valid(cls, action):
        return action in cls._value2member_map_

    def is_moving_action(self):
        return self.value in [self.MOVE_RIGHT, self.MOVE_LEFT, self.MOVE_FORWARD, self.REVERSE]


RailEnvGridPos = NamedTuple('RailEnvGridPos', [('r', int), ('c', int)])
RailEnvNextAction = NamedTuple('RailEnvNextAction', [('action', RailEnvActions), ('next_position', RailEnvGridPos),
                                                     ('next_direction', Grid4TransitionsEnum)])

import numpy as np

from flatland.core.grid.grid_utils import IntVector2D, IntVector2DDistance
from flatland.core.grid.grid_utils import IntVector2DArray
from flatland.core.grid.grid_utils import Vec2dOperations as Vec2d
from flatland.core.transition_map import GridTransitionMap
from flatland.utils.ordered_set import OrderedSet


class AStarNode:
    """A node class for A* Pathfinding"""

    def __init__(self, pos: IntVector2D, parent=None):
        self.parent = parent
        self.pos: IntVector2D = pos
        self.g = 0.0
        self.h = 0.0
        self.f = 0.0

    def __eq__(self, other):
        """

        Parameters
        ----------
        other : AStarNode
        """
        return self.pos == other.pos

    def __hash__(self):
        return hash(self.pos)

    def update_if_better(self, other):
        if other.g < self.g:
            self.parent = other.parent
            self.g = other.g
            self.h = other.h
            self.f = other.f


def a_star(grid_map: GridTransitionMap, start: IntVector2D, end: IntVector2D,
           a_star_distance_function: IntVector2DDistance = Vec2d.get_manhattan_distance, avoid_rails=False,
           respect_transition_validity=True, forbidden_cells: IntVector2DArray = None) -> IntVector2DArray:
    """

    :param avoid_rails:
    :param grid_map: Grid Map where the path is found in
    :param start: Start positions as (row,column)
    :param end:  End position as (row,column)
    :param a_star_distance_function: Define the distance function to use as heuristc:
            -get_euclidean_distance
            -get_manhattan_distance
            -get_chebyshev_distance
    :param respect_transition_validity: Whether or not a-star respect allowed transitions on the grid map.
            - True: Respects the validity of transition. This generates valid paths, of no path if it cannot be found
            - False: This always finds a path, but the path might be illegal and thus needs to be fixed afterwards
    :param forbidden_cells: List of cells where the path cannot pass through. Used to avoid certain areas of Grid map
    :return: IF a path is found a ordered list of al cells in path is returned
    """
    """
    Returns a list of tuples as a path from the given start to end.
    If no path is found, returns path to closest point to end.
    """
    rail_shape = grid_map.grid.shape

    start_node = AStarNode(start, None)
    end_node = AStarNode(end, None)
    open_nodes = OrderedSet()
    #print('open nodes all inizio',open_nodes)
    closed_nodes = OrderedSet()
    open_nodes.add(start_node)

    while len(open_nodes) > 0:
        # get node with current shortest est. path (lowest f)
        current_node = None
        for item in open_nodes:
            if current_node is None:
                current_node = item
                #print('Current node all inizio',current_node)
                continue
            if item.f < current_node.f:
                current_node = item
                #print('Current node in generale',current_node.pos, current_node.parent.pos, current_node.f)

            #print('open nodes nel ciclo ', open_nodes.pos)

        # pop current off open list, add to closed list
        open_nodes.remove(current_node)
        closed_nodes.add(current_node)

        # found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.pos)
                current = current.parent

                #print(path)

            # return reversed path
            return path[::-1]

        # generate children
        children = []
        if current_node.parent is not None:
            prev_pos = current_node.parent.pos
        else:
            prev_pos = None

        for new_pos in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            # update the "current" pos
            node_pos: IntVector2D = Vec2d.add(current_node.pos, new_pos)

            # is node_pos inside the grid?
            if node_pos[0] >= rail_shape[0] or node_pos[0] < 0 or node_pos[1] >= rail_shape[1] or node_pos[1] < 0:
                continue

            # validate positions
            #

            if not grid_map.check_transition_is_possible(prev_pos, current_node.pos, node_pos) \
             and respect_transition_validity:
                continue

            '''
            if grid_map.validate_new_transition(prev_pos, current_node.pos, node_pos, end_node.pos) and respect_transition_validity:
                   # and grid_map.check_direction_of_railroad(prev_pos, current_node.pos, node_pos):
                print('=========================================')
                print(prev_pos, current_node.pos, node_pos)
                continue
            '''


            # create new node

            new_node = AStarNode(node_pos, current_node)
            #print(new_node.pos)

            # Skip paths through forbidden regions if they are provided
            if forbidden_cells is not None:
                if node_pos in forbidden_cells and new_node != start_node and new_node != end_node:
                    continue

            children.append(new_node)

        # loop through children
        for child in children:
            # already in closed list?
            if child in closed_nodes:
                continue

            # create the f, g, and h values
            child.g = current_node.g + 1.0
            # this heuristic avoids diagonal paths
            if avoid_rails:
                child.h = a_star_distance_function(child.pos, end_node.pos) + np.clip(grid_map.grid[child.pos], 0, 1)
            else:
                child.h = a_star_distance_function(child.pos, end_node.pos)
            child.f = child.g + child.h

            # already in the open list?
            if child in open_nodes:
                continue

            # add the child to the open list
            open_nodes.add(child)

        # no full path found
        if len(open_nodes) == 0:
            return []


# Flatland Custom Rail Generator

--- 
A custom rail is a rail defined by a matrix. Each cell of the matrix define the type of cell in the 2-D grid of the flatland environment



## Different type of cell

<img src="https://i.imgur.com/ruiRuep.jpg" width="500"/> <img src="https://i.imgur.com/sABiSuc.jpg" width="500"/>

* This is the function to create a custom rail

In [10]:
"""Rail generators (infrastructure manager, "Infrastrukturbetreiber")."""
from typing import Callable, Tuple, Optional, Dict, List

from numpy.random.mtrand import RandomState

from flatland.core.grid.grid4 import Grid4TransitionsEnum
from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transition_map import GridTransitionMap


RailGeneratorProduct = Tuple[GridTransitionMap, Optional[Dict]]
""" A rail generator returns a RailGenerator Product, which is just
    a GridTransitionMap followed by an (optional) dict/
"""

RailGenerator = Callable[[int, int, int, int], RailGeneratorProduct]

# Number of agents are given by timetables (the num of the rows), target stations are also given by timetable 
def rail_custom_generator(rail_spec, train_stations_position: list = None, timetable: list = None):

    """
    Utility to convert a rail given by manual specification as a map of tuples
    (cell_type, rotation), to a transition map with the correct 16-bit
    transitions specifications.

    Parameters
    ----------
    rail_spec : list of list of tuples
        List (rows) of lists (columns) of tuples, each specifying a rail_spec_of_cell for
        the RailEnv environment as (cell_type, rotation), with rotation being
        clock-wise and in [0, 90, 180, 270].

    Returns
    -------
    function
        Generator function that always returns a GridTransitionMap object with
        the matrix of correct 16-bit bitmaps for each rail_spec_of_cell.

    New features:
    -------------
        Train station poition: to define the position of the station
        Target station: Define the target of the different agents
        Timetable: timetable contein the intermediate station and the time at which pass across them
    """

    def custom_generator(width: int, height: int, num_agents: int, num_resets: int = 0,
                  np_random: RandomState = None) -> RailGenerator:

        # All the cities are oriented in the same way in my model
        city_orientations = 1

        # Taking the number of agents from timetable
        num_of_agents = len(timetable)

        # Taking the target stations from timetable
        target_stations = []

        for agent in range(num_agents):
            target_stations.append(timetable[agent][-1])
      
        rail_env_transitions = RailEnvTransitions()

        height = len(rail_spec)
        width = len(rail_spec[0])
        rail = GridTransitionMap(width=width, height=height, transitions=rail_env_transitions)

        for r in range(height):
            for c in range(width):
                rail_spec_of_cell = rail_spec[r][c]
                index_basic_type_of_cell_ = rail_spec_of_cell[0]
                rotation_cell_ = rail_spec_of_cell[1]
                if index_basic_type_of_cell_ < 0 or index_basic_type_of_cell_ >= len(rail_env_transitions.transitions):
                    print("ERROR - invalid rail_spec_of_cell type=", index_basic_type_of_cell_)
                    return []
                basic_type_of_cell_ = rail_env_transitions.transitions[index_basic_type_of_cell_]
                effective_transition_cell = rail_env_transitions.rotate_transition(basic_type_of_cell_, rotation_cell_)
                rail.set_transitions((r, c), effective_transition_cell)

        return rail,  {'agents_hints': {
            'num_agents': num_of_agents,            
            'city_positions': train_stations_position,
            'train_stations': train_stations_position,
            'city_orientations': city_orientations,
            'targets' : target_stations,
            'timetable' : timetable 
        }}

    return custom_generator


* these are the utilities for the environment


In [11]:
from flatland.core.env_observation_builder import ObservationBuilder
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.predictions import ShortestPathPredictorForRailEnv
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import rail_from_file
from flatland.envs.line_generators import line_from_file
from flatland.envs.step_utils.states import TrainState


def load_flatland_environment_from_file(file_name: str,
                                        load_from_package: str = None,
                                        obs_builder_object: ObservationBuilder = None,
                                        record_steps = False,
                                        ) -> RailEnv:
    """
    Parameters
    ----------
    file_name : str
        The pickle file.
    load_from_package : str
        The python module to import from. Example: 'env_data.tests'
        This requires that there are `__init__.py` files in the folder structure we load the file from.
    obs_builder_object: ObservationBuilder
        The obs builder for the `RailEnv` that is created.


    Returns
    -------
    RailEnv
        The environment loaded from the pickle file.
    """
    if obs_builder_object is None:
        obs_builder_object = TreeObsForRailEnv(
            max_depth=2,
            predictor=ShortestPathPredictorForRailEnv(max_depth=10))
    environment = RailEnv(width=1, height=1, rail_generator=rail_from_file(file_name, load_from_package),
                          line_generator=line_from_file(file_name, load_from_package),
                          number_of_agents=1,
                          obs_builder_object=obs_builder_object,
                          record_steps=record_steps,
                          )
    return environment

# TODO riferisciti a un agente (EnvAgent)
# Usa le azioni dell'agente specifico
def delay_a_train(delay: int, delay_time: int, time_of_train_generation: int, actions, train):
    
    i_agent = train.handle
    train_velocity = train.speed_counter.speed

    
    actions_scheduled = [0] * (len(actions[i_agent]) + delay)
    
    # Copy the actions scheduled for the train before the delay
    for i in range(delay_time - time_of_train_generation):
        actions_scheduled[i] = actions[i_agent][i]
    # Delay the train (STOP)  
    for i in range(delay):
        if (i + delay_time - time_of_train_generation) < 0 or (delay_time - time_of_train_generation) > len(actions[i_agent]):
            raise ImportError('The train is not present in the environment, check the delay time')
        actions_scheduled[i + delay_time - time_of_train_generation] = RailEnvActions.STOP_MOVING
    # Copy the actions scheduled for the train after the delay
    for i in range(len(actions[i_agent]) - (delay_time - time_of_train_generation)):
        actions_scheduled[i + delay_time - time_of_train_generation + delay] = actions[i_agent][i + delay_time - time_of_train_generation]
    
    actions[i_agent] = actions_scheduled
    return 


# Function to convert decimal number to base number
def actions_decimal_to_base(base, number_to_convert, num_agents):
    division = number_to_convert
    result = []
    while division != 0:
        result.append(division % base)
        division = int(division / base)
    actions_to_perform = result[::-1]
    if len(actions_to_perform) < num_agents:
        zero_to_add = num_agents - len(actions_to_perform)
        for i in range(zero_to_add):
            actions_to_perform.insert(0,0)
    return actions_to_perform


def make_a_deterministic_interruption(agent_to_interrupt, interruption_time):
    if agent_to_interrupt.state.is_on_map_state():
        agent_to_interrupt.malfunction_handler.malfunction_down_counter = interruption_time



# Custom schedule generator


In [12]:
"""Schedule generators (railway undertaking, "EVU")."""
from typing import Tuple, List, Callable, Mapping, Optional, Any

from enum import IntEnum
from numpy.random.mtrand import RandomState
 
from flatland.core.transition_map import GridTransitionMap
from typing import List, NamedTuple, Callable

from flatland.core.grid.grid4 import Grid4TransitionsEnum
from flatland.core.grid.grid_utils import IntVector2DArray


Schedule = NamedTuple('Schedule', [('agent_positions', IntVector2DArray),
                                   ('agent_directions', List[Grid4TransitionsEnum]),
                                   ('agent_targets', IntVector2DArray),
                                   ('agent_speeds', List[float]),
                                   ('agent_malfunction_rates', List[int]),
                                   ('max_episode_steps', int)]) 

def check_rail_road_direction(rail: GridTransitionMap, timetable):
    # To establish the direction of trains in the railroas I define a simple law, as for the cars, each trains has to 
    # go the direction that let them to have the right free

    agents_direction = [0]*len(timetable)
    path_result = [0]*len(timetable)

    #print(rail.grid[7 ,13],rail.grid[7 ,12],rail.grid[7 ,11],rail.grid[7 ,10],rail.grid[7 ,9],rail.grid[7 ,8],rail.grid[7 ,7],rail.grid[7 ,6], rail.grid[7 ,5])
    
    for i in range (len(timetable)):
        # Consider the a_star result to calculate the direction
        path_result[i] = (a_star(rail,timetable[i][0][0],timetable[i][0][1]))
        if path_result[i] == []:
            agents_direction[i] = 1
            continue
        if len(path_result[i]) == 1:
            agents_direction[i] = 1
            continue
        difference_x = path_result[i][0][1] - path_result[i][1][1]
        difference_y = path_result[i][0][0] - path_result[i][1][0]
        if difference_y == 1:
            agents_direction[i] = 0
        if difference_x ==  -1:
            agents_direction[i] = 1
        if difference_y == -1:
            agents_direction[i] = 2
        if difference_x == 1:
            agents_direction[i] = 3

    return agents_direction

AgentPosition = Tuple[int, int]
ScheduleGenerator = Callable[[GridTransitionMap, int, Optional[Any], Optional[int]], Schedule]


def custom_schedule_generator(timetable, speed_ratio_map: Mapping[float, float] = None, seed: int = 1) -> ScheduleGenerator:

#class Custom_schedule_generator(BaseSchedGen):
	"""

	This is a custom schedule generator, create a schedule with the timetable, and the station where the trains should pass
	"""

	def generate_custom(rail: GridTransitionMap, num_agents: int, hints: Any = None, num_resets: int = 0,
				  np_random: RandomState = None) -> Schedule:
		"""

		The generator that assigns tasks to all the agents
		:param rail: Rail infrastructure given by the rail_generator
		:param num_agents: Number of agents to include in the schedule
		:param hints: Hints provided by the rail_generator These include positions of start/target positions
		:param num_resets: How often the generator has been reset.
		:return: Returns the generator to the rail constructor
		"""

		train_stations = hints['train_stations']
		city_positions = hints['city_positions']
		city_orientation = hints['city_orientations']
		max_num_agents = hints['num_agents']

		if num_agents > max_num_agents:
			num_agents = max_num_agents
			warnings.warn("Too many agents! Changes number of agents.")
		# Place agents and targets within available train stations
		agents_position = []
		agents_target = []
		agents_direction = []

		# Define the agent positions
		# TODO capisci se così va bene o no
		for agent_i in range (len(timetable)):
			agents_position.append(timetable[agent_i][0][0])
			agents_target.append(timetable[agent_i][0][-1])


		# Define the direction of the trains based on the rail they occupy
		# Input --> the topology of the network, the position of the trains
		# Output --> an array with the directions of the trains
		# DIRECTIONS: 0 = UP, 1 = RIGHT, 2 = DOWN, 3 = LEFT

		agents_direction = check_rail_road_direction(rail, timetable)


		_runtime_seed = seed + num_resets

		if speed_ratio_map:
			speeds = speed_initialization_helper(num_agents, speed_ratio_map, seed=_runtime_seed, np_random=np_random)
		else:
			speeds = [1.0] * len(agents_position)

		# We add multiply factors to the max number of time steps to simplify task in Flatland challenge.
		# These factors might change in the future.
		timedelay_factor = 4
		alpha = 2
		max_episode_steps = 1000

		#print(agents_position, agents_target, agents_direction)

		return Schedule(agent_positions=agents_position, agent_directions=agents_direction,
						agent_targets=agents_target, agent_speeds=speeds, agent_malfunction_rates=None,
						max_episode_steps=max_episode_steps)

	return generate_custom  #(station_to_traverse = [(21, 37), (15, 51)])

* These are the plan to follow utils

In [13]:
# import chain
from itertools import chain
from enum import IntEnum
import random
# This is to test if the timetable is valid or not
from flatland.core.grid.grid4_astar import a_star

class RailEnvActions(IntEnum):
    DO_NOTHING = 0  # implies change of direction in a dead-end!
    MOVE_LEFT = 1
    MOVE_FORWARD = 2
    MOVE_RIGHT = 3
    STOP_MOVING = 4
    REVERSE = 5

    @staticmethod
    def to_char(a: int):
        return {
            0: 'B',
            1: 'L',
            2: 'F',
            3: 'R',
            4: 'S',
            5: 'I'
        }[a]

# Check if the timetable is feaseble or not
def control_timetable(timetable, railway_topology):
    # Check for all the trains
    for trains in range (len(timetable)):       
        # Check for all the stations
        # Calculate the difference of two different times, so i don't need the last term to cycle          
        for stations in range (len(timetable[trains][1]) - 1):   
            if (timetable[trains][1][stations] - timetable[trains][1][stations + 1]) >= 0:
                print('===================================================================================================================================')
                print('Attention!!! The agent number', trains, 'has a problem in the timetable, times to reach stations', stations, 'and', (stations+1), 'are not right')
                print('The time to reach the successive station SHOULD BE > 0, pay attenction to the timetable')
                # Function that check if the time to reach a station defined by the timetable are possible or not,
                # Return the time minimum time to reach two different stations depending on the distance and on the line type (high velocity, regional...)
            time_to_next_station = time_to_reach_next_station(timetable[trains][0][stations], timetable[trains][0][stations + 1], railway_topology, timetable, trains)
            # Control if the time to reach the next station is possible (considering maximum velocities of lines and the distances between two stations)
            if time_to_next_station > (timetable[trains][1][stations+1]- timetable[trains][1][stations]):
                print('===================================================================================================================================')
                print('Attention!!! Agent number', trains, 'has a problem in the timetable, times to reach stations', stations, 'and', (stations+1), 'are not right')
                print('The time to reach the next station SHOULD BE HIGHER. The minimum time to reach the station should be:', time_to_next_station)
    return

# TODO aggiungere dei controlli 
# - controllare che la posizione in cui si mette il goal del secondo treno sia possibile
# - controllare il tempo necessario nella stazione (in base anche alla rialzatura in cui si mette il treno 
#   e.g. treno spostato su di due caselle, tempo necessario + 4 * velocità alla meno uno)

def divide_trains_in_station_rails(timetable, railway_topology):
    # The number of different stations presented in the timetable
    different_stations = 0
    # The position of different stations presented in the timetable
    station_positions = []
    # Check how many different stations are in the timetable
    for i in range(len(timetable)): # for all the trains
        for k in range(len(timetable[i][0])): # for all the stations
            if timetable[i][0][k] not in station_positions:
                station_positions.append(timetable[i][0][k])
                different_stations += 1

    # Indexes contein the indexes of the station like this
    # 0 for the first station in station position 
    # 1 for the second and so on
    indexes = []
    # Local variable for the loop
    single_index = []

    for j in range(len(timetable)): # for all the trains
        single_index = [0]*len(timetable[j][0])
        for k in range(len(timetable[j][0])): # for all the stations
            for i in range(len(station_positions)): # for all the different stations discovered
                if station_positions[i] == timetable[j][0][k]:
                    single_index[k] = i
        indexes.append(single_index)
    
    # Stations have a capacity, if more trains with respect to the capacity
    # of the station are present there is a problem
    counter_of_trains = [0] * different_stations  # counter of trains in a certain station
    
    # TODO fai un ciclo for che cicli gli step nella tabella oraria per aggiornare il counter trains
    # quando più treni sono nella stessa stazione aumenta, quando un treno esce dalla stazione diminuisce
    # aggiungi una lettera che cicli sulle due stazioni	(ad ora le stazioni girano su tutta la timetable)
    # Calculating the maximum time the agents have to stay in env 
    
     # Here controll if two or more trains have to reach the same station at the same time
    for m in range(len(timetable)):  # for all the trains m
        for n in range(len(timetable)): # for all the trains n with m!=n
            for k in range(len(timetable[m][1])): # for all the stations of m
                for l in range(len(timetable[n][1])): # for all the stations of n
                    if m!=n and m<n:
                        # time at which the trains have to reach the stations
                        time_train_a = timetable[m][1][k]
                        time_train_b = timetable[n][1][l]
                        # if they have to reach the same stations
                        if indexes[m][k] == indexes[n][l]: 
                            threshold_time = calculate_time_in_station(timetable,m,n,k,l)
                            # if the time at which they have to reach the station is similar (calculated the time needed in the stations for the trains)
                            if time_train_a - time_train_b < threshold_time and time_train_a - time_train_b > -threshold_time:
                                # I change the goal of the convoy that first reach the station
                                timetable[n][0][l] = (station_positions[indexes[n][l]][0] - 1, station_positions[indexes[n][l]][1])
                            


# Define the scheduled actions the agents have to do
def action_to_do(timetable, railway_topology):
    # Path to do to arrive to the right station

    # L'idea vuole essere quella di avere un tempo necessario per stare in stazione
    # Se due treni nell'intorno di tempo devono stare nella stessa stazione bisogna indirizzarli 
    # in binari liberi diversi, quindi il passaggio diventa diverso.

    path_result = []
    # Calculate the path for all the trains
    for train_i in range (len(timetable)):
        # Number of stations in the train i
        num_of_stations = len(timetable[train_i][0])
        # The partial result a train run
        path_partial_result = []
        for station in range(num_of_stations - 1): 
            path_partial_result.append(a_star(railway_topology,timetable[train_i][0][station],timetable[train_i][0][station + 1]))
            if path_partial_result == []:
                raise ImportError('There s not a path between station', station, 'and station', station + 1 )

        # Final result for all the trains and train runs
        path_result.append(path_partial_result)

    # Calculate the actions that have to be done
    actions_to_do = []
    for train_i in range (len(timetable)):
        # Number of stations in the train i
        num_of_stations = len(timetable[train_i][0])
        # Flag that tells me that the next step is particular
        next = False
        # Each train occupy a row in the action_to_do matrix 
        actions_single_train = []
        # I need the direction of the last run for the reverse action
        direction_last_run = 0
        for station in range (num_of_stations - 1):
            # Each train occupy a row in the action_to_do matrix 
            actions_single_train_run = []
            for step in range (len(path_result[train_i][station])):
                # If i'm restarting from the final station of the train run, i have to wait till is the time to restart
                if len(path_result[train_i][station]) == 1:
                    time_to_wait = timetable[train_i][1][station + 1] - timetable[train_i][1][station]
                    for i in range(time_to_wait):
                        actions_single_train_run.append(RailEnvActions.STOP_MOVING)
                    continue
                # Calculate the direction of the trains at each step
                if step == 0:
                    difference_y = path_result[train_i][station][step][0] - path_result[train_i][station][step + 1][0]
                    difference_x = path_result[train_i][station][step][1] - path_result[train_i][station][step + 1][1]
                    if difference_y == 1:
                        direction = 0
                    if difference_x ==  -1:
                        direction = 1
                    if difference_y == -1:
                        direction = 2
                    if difference_x == 1:
                        direction = 3 
                else:
                    difference_y = path_result[train_i][station][step - 1][0] - path_result[train_i][station][step][0]
                    difference_x = path_result[train_i][station][step - 1][1] - path_result[train_i][station][step][1]
                    if difference_y == 1:
                        direction = 0
                    if difference_x ==  -1:
                        direction = 1
                    if difference_y == -1:
                        direction = 2
                    if difference_x == 1:
                        direction = 3 
                # Variable to count the number of possible path at each cell, is an int with the number of possible path
                if not step == 0:
                    # Specific case, a train is at the boarder of two different lines, 
                    # if this appen I have to consider the previous transition at the next time stamp due to the fact the velocity changes
                    if next:
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step-1][0],path_result[train_i][station][step-1][1],prev_direction).count(1)
                        next = False
                    elif (path_result[train_i][station][step] in av_line) and not (path_result[train_i][station][step - 1] in av_line):
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step][0],path_result[train_i][station][step][1],prev_direction).count(1)
                        next = True
                    else:
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step-1][0],path_result[train_i][station][step-1][1],prev_direction).count(1)
                # Starting with a move forward direction for the train
                if step == 0:
                    #actions_single_train.append(RailEnvActions.MOVE_FORWARD)
                    prev_direction = direction
                # If I'm not at the start of the train 
                else:
                    # The direction doesn't change
                    if num_of_stations > 1 and station >= 1 and step <= 1:
                        if (direction - direction_last_run == 2 or direction - direction_last_run == -2):
                            # If I'm in an hig velocity line velocity is define only by the type of train
                            actions_single_train_run.append(RailEnvActions.REVERSE)
                            prev_direction = direction
                            continue

                    if direction - prev_direction == 0:
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            #print('Test per capire come varia',i, 'Treno numero', train_i)
                            actions_single_train_run.append(RailEnvActions.MOVE_FORWARD)
                        # I'm arrived at the station?
                        if step == (len(path_result[train_i][station]) - 1):
                            # If the next station is the last one of the train run I don't have to stop for the min wait time
                            if station != (num_of_stations - 2):
                                if len(path_result[train_i][station + 1]) == 1:
                                    continue
                            # If is an intermediate station I need to stop the min wait time
                            for i in range(3):   # TODO ADD THE WAITING TIMES OF THE STATIONS
                                actions_single_train_run.append(RailEnvActions.STOP_MOVING)

                        prev_direction = direction
                    # I have to reverse the train direction when I arrive at the ending station
                    elif ((direction - prev_direction) == -2) or ((direction - prev_direction) == 2):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.REVERSE)
                        prev_direction = direction
                    # I have to move to left 
                    # and I have more then one possible path, so I go left at the deviation
                    # Depending on the direction of march the results can be -1 or -3
                    elif ((direction - prev_direction == -1) and (multiple_path > 1)) or ((direction - prev_direction == +3)):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_LEFT)
                        prev_direction = direction
                    # I have to move right 
                    # and I have more then one possible path, so I go left at the deviation 
                    # Depending on the direction of march the results can be +1 or -3
                    elif ((direction - prev_direction == 1) and (multiple_path > 1)) or ((direction - prev_direction == -3) ):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_RIGHT)
                        prev_direction = direction
                    else:
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_FORWARD)
                        # I'm arrived at the station?
                        if step == (len(path_result[train_i][station]) - 1):
                            for i in range(3):   # TODO ADD THE WAITING TIMES OF THE STATIONS
                                actions_single_train_run.append(RailEnvActions.STOP_MOVING)
                        prev_direction = direction
            direction_last_run = direction

            actions_single_train.append(actions_single_train_run)
 
        if isinstance(actions_single_train[0], list):
            len(actions_single_train)
            actions_single_train = list(chain.from_iterable(actions_single_train))
        actions_to_do.append(actions_single_train)

    return actions_to_do

# Calculate the time to reach the stations to understand if timetable is right
def time_to_reach_next_station(departure_station_position, arrival_station_position, railway_topology, schedule, train_number):
    # First thing check the distance between two stations 
    result = a_star(railway_topology, departure_station_position, arrival_station_position)
    # Maximum velocity a train can achieve
    train_velocity = schedule[train_number][2]

    lenght_path = len(result)  # distance between stations

    # Array when I put at each step the time needed to make the path
    # The total time is the sum of the numbers
    time_array = []
    # Check the at each step which train i am and which line im in
    for step in range(lenght_path):
        if (result[step]) in av_line:
            time_array.append(pow(train_velocity,-1))
        else:
            time_array.append(pow(min(train_velocity, 1/2), -1))
    time_needed = sum(time_array)

    #print((time_needed + int(time_needed/10))) DEBUG

    # Adding to the time a 10% to face with problems in case it's neaded
    return (time_needed + int(time_needed/10))

# TODO calculate the av_rails, in order to distinguish them
# TODO calculate the right,left,up,down rails, in order to distinguish them
# TODO understad if the velocities are realistic or not (360 km for high velocity, 180 and 120 is realistic or not?)

def calculate_timetable(convoys, railway_topology):
    # The timetable that should be returned
    timetable = []
    # For each convoy
    for convoy_i in range(len(convoys)):
        # For each train run defined
        single_convoy_schedule = convoys[convoy_i].schedule
        single_convoy_schedule_len = len(convoys[convoy_i].schedule)
        single_convoy = []
        for num_of_runs in range(single_convoy_schedule_len):
            # The single train run
            single_train_run = []
            # The number of station to pass
            num_of_stations = len(single_convoy_schedule[num_of_runs].line_belongin.stations)
            # The station to stop
            stations_to_stop_position = []
            # Direction not inverted?
            if not single_convoy_schedule[num_of_runs].inverse_train_direction:
                for i in range(num_of_stations):
                    # append the station position in the right order
                    stations_to_stop_position.append(single_convoy_schedule[num_of_runs].line_belongin.stations[i].position)
            # Direction inverdet?
            else:
                for i in range(num_of_stations):
                    # append the station position in inverted order
                    stations_to_stop_position.append(single_convoy_schedule[num_of_runs].line_belongin.stations[num_of_stations - 1 - i].position)
            # Adding the starting time
            single_train_run.append(single_convoy_schedule[num_of_runs].starting_time)

            for stations in range(num_of_stations -1):
                departure_station_position = stations_to_stop_position[stations]
                arrival_station_position = stations_to_stop_position[stations + 1]
                # First thing check the distance between two stations
                result = a_star(railway_topology, departure_station_position, arrival_station_position)
                # Maximum velocity a train can achieve
                train_velocity = convoys[convoy_i].maximum_velocity 

                lenght_path = len(result)  # distance between stations

                # Array when I put at each step the time needed to make the path
                # The total time is the sum of the numbers
                time_array = []

                # Check the at each step which train i am and which line im in
                # Train should be in the middle of two line type
                for step in range(lenght_path):
                    if (result[step]) in av_line:
                        time_array.append(pow(train_velocity,-1))
                    else:
                        time_array.append(pow(min(train_velocity, 1/2), -1))
                time_needed = sum(time_array)
                # Adding to the time a 10% to face with problems in case it's neaded
                time_needed = time_needed + int(time_needed/10)

                # Adding the precedence time 
                if stations != (num_of_stations - 1):
                    if len(single_train_run) == 1 and type(single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time) == int:
                        single_train_run.append(int(time_needed + single_train_run[0] + single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time))
                    else:
                        # sum of time needed, the precedence time and the waiting time at the station
                        single_train_run.append(int(time_needed + single_train_run[stations] + single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time[0]))

            single_convoy.append(stations_to_stop_position)
            single_convoy.append(single_train_run)
            
        timetable.append(single_convoy)

    # This is needed in order to obtein the timetable-standard structure
    final_timetable = [] # The final timetable
    timetable_single_convoy = [] # Timetable of a single convoy
    timetable_position_example = [] # Partial timetable with the position of the stations to pass
    timetable_time_example = []  # Partial timetable with the time at which reach the stations
    single_position_timetable = []  # position and time of a single train run
    single_time_timetable = []

    for i in range(len(timetable)):
        for j in range(len(timetable[i])):
            if j % 2 == 0:
                timetable_position_example.append(timetable[i][j])
            else:
                timetable_time_example.append(timetable[i][j])
        for k in range(len(timetable_position_example)):
            single_position_timetable += (timetable_position_example[k])
        for k in range(len(timetable_time_example)):
            single_time_timetable += timetable_time_example[k]
        # The standard timetable form is (positions, time, train velocity)
        timetable_single_convoy.append(single_position_timetable)
        timetable_single_convoy.append(single_time_timetable)
        timetable_single_convoy.append(convoys[i].maximum_velocity)
        # Final timetable
        final_timetable.append(timetable_single_convoy)
        # Restart the partial results
        single_position_timetable = []
        timetable_position_example = []
        single_time_timetable = []
        timetable_time_example = []
        timetable_single_convoy = []

    return final_timetable

def calculate_time_in_station(timetable,train_a,train_b,index_a,index_b):
    time_a = 15
    time_b = 15
    if index_a != 0:
        time_a = timetable[train_a][1][index_a] - timetable[train_a][1][index_a - 1] + 15
    if index_b != 0:
        time_b = timetable[train_b][1][index_b] - timetable[train_b][1][index_b - 1] + 15
    return max(time_a,time_b)

# Rail Environment

## Configuration

In [14]:
from typing import Type
import numpy as np
import os
# Import the structures


example_training = 'training0'
'''
###############################################################
######################   EXAMPLE 1  #########################
###############################################################
'''

if example_training == 1:
	# Import the examples
	from examples.new_example_1 import rail, railway_example, av_line
	# Define the stations
	quarto_station = Station('Quarto', position = (3, 2), capacity = 2, min_wait_time = [2, 2, 1], 
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	quinto_station = Station('Quinto', position = (3, 9), capacity = 2, min_wait_time = [2, 2, 1], 
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)

	# Define the rail connection beetween the two stations
	connection_quarto_quinto = Rail_connection(station_a = quarto_station, 
		station_b = quinto_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	# Define the lines
	genova_urbana = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (quarto_station, quinto_station), stops = (1, 1))

	stations = []
	stations.append([quarto_station.position, 0.5])
	stations.append([quinto_station.position, 0.5])

	stations_objects = [quarto_station, quinto_station]

	# Define the train runs
	train_run_0 = Train_run(genova_urbana, starting_time = 3, from_depot = True)
	train_run_1 = Train_run(genova_urbana, starting_time = 10, from_depot = True, inverse_train_direction = True)
	train_run_2 = Train_run(genova_urbana, starting_time = 40, inverse_train_direction = True)
	train_run_3 = Train_run(genova_urbana, starting_time = 70)

	# Define the convoys
	R1079_convoy = Convoy( Type_of_convoy.INTERCITY)
	R1078_convoy = Convoy( Type_of_convoy.INTERCITY)

	convoys = [R1079_convoy, R1078_convoy]

	# Adding the train runs to the convoys
	R1079_convoy.add_train_run(train_run_0)
	R1079_convoy.add_train_run(train_run_2)
	R1078_convoy.add_train_run(train_run_1)
	R1078_convoy.add_train_run(train_run_3)

	# Generating the timetable
	# The timetable is composed by (station positions, time at which reach the stations, maximum train velocity)
	timetable_example = calculate_timetable(convoys, rail)
""" 
	# TODO crea una funzione (per ora non è una funzione, serve quella) per esportare le timetable come excel
	from pandas import DataFrame
	import pandas as pd

	df_quarto = [0]* len(convoys)
	df_quinto = [0]* len(convoys)
	df = [0]* len(convoys)
	quarto = []
	quinto = []
	for i in range(len(convoys)):
		for j in range(len(stations_objects)):
			for k in range(len(timetable_example[i][0])):
				if j == 0:
					if timetable_example[i][0][k] == stations_objects[j].position:
						quarto.append(timetable_example[i][1][k])
				if j == 1:
					if timetable_example[i][0][k] == stations_objects[j].position:
						quinto.append(timetable_example[i][1][k])
		df_quarto[i] = DataFrame({stations_objects[0].name : quarto})
		df_quinto[i]  = DataFrame({stations_objects[1].name : quinto})
		frames = [df_quarto[i], df_quinto[i]]
		df[i] = pd.concat(frames, axis=1)
		quarto = []
		quinto = []

		# funtion
	def multiple_dfs(df_list, sheets, file_name, spaces):
	    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')   
	    row = 0
	    for dataframe in df_list:
	        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=0)   
	        row = row + len(dataframe.index) + spaces + 1
	    writer.save()

	# run function
	os.makedirs("output/timetables", exist_ok=True)
	multiple_dfs(df, 'Validation', 'output/timetables/timetable_test_1.xlsx', 1)"""

if example_training == 2:
	# Import the examples
	from examples.new_example_2 import rail, railway_example, av_line

	# Define the stations
	# TODO raagiona sul come definire meglio la capacità delle stazioni e gestire i binari....
	quarto_station = Station('Quarto', position = (6, 2), capacity = 5, min_wait_time = [2, 2, 1], 
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	quinto_station = Station('Quinto', position = (6, 17), capacity = 5, min_wait_time = [2, 2, 1], 
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)


	# Define the rail connection beetween the two stations
	connection_quarto_quinto = Rail_connection(station_a = quarto_station, 
		station_b = quinto_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])


	# Define the lines
	genova_urbana = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (quarto_station, quinto_station), stops = (1, 1))

	stations = []
	stations.append([quarto_station.position, 0.5])
	stations.append([quinto_station.position, 0.5])
	stations_objects = [quarto_station, quinto_station]

	# Define the train runs
	train_run_0 = Train_run(genova_urbana, starting_time = 3, from_depot = True)
	train_run_1 = Train_run(genova_urbana, starting_time = 40, from_depot = True, inverse_train_direction = True)
 
	train_run_2 = Train_run(genova_urbana, starting_time = 55, inverse_train_direction = True)
 
	train_run_3 = Train_run(genova_urbana, starting_time = 20, from_depot = True)
	train_run_5 = Train_run(genova_urbana, starting_time = 70, inverse_train_direction = True, to_depot = True)
 
	train_run_4 = Train_run(genova_urbana, starting_time = 20, from_depot = True)
	train_run_6 = Train_run(genova_urbana, starting_time = 75, inverse_train_direction = True, to_depot = True)

	# Define the convoys
	R1079_convoy = Convoy( Type_of_convoy.INTERCITY)
	R1078_convoy = Convoy( Type_of_convoy.INTERCITY)
	R1077_convoy = Convoy( Type_of_convoy.INTERCITY)

	# SOME MODIFICATION TO STUDY TRAINING
	#R1076_convoy = Convoy( Type_of_convoy.INTERCITY)

	convoys = [R1079_convoy, R1078_convoy, R1077_convoy] #, R1076_convoy]

	# Adding the train runs to the convoys
	# R1079
	R1079_convoy.add_train_run(train_run_0)
	R1079_convoy.add_train_run(train_run_2)
	# R1078
	R1078_convoy.add_train_run(train_run_1)
	# R1077
	R1077_convoy.add_train_run(train_run_3)
	R1077_convoy.add_train_run(train_run_5)
	# R1076
	#R1076_convoy.add_train_run(train_run_4)

	# Generating the timetable
	# The timetable is composed by (station positions, time at which reach the stations, maximum train velocity)
	timetable_example = calculate_timetable(convoys, rail)

'''
###############################################################
######################   EXAMPLE 3  #########################
###############################################################
'''

if example_training == 3:
	# Import the examples
	from examples.new_example_3 import rail, railway_example, av_line

	# Define the stations
	genova_station = Station('Genova', position = (21,2), capacity = 2, min_wait_time = [2, 2, 1], 
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	pavia_station = Station('Pavia', position = (21,18), capacity = 2, min_wait_time = [2, 2, 1],
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	milano_station = Station('Milano', position = (15,49), capacity = 2, min_wait_time = [2, 2, 1],
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	torino_station = Station('Torino', position = (9,1), capacity = 2, min_wait_time = [2, 2, 1],
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)
	novara_station = Station('Novara', position = (3,40), capacity = 2, min_wait_time = [2, 2, 1],
		additional_wait_percent = [0.5, 1, 1.5], importance = 80, railway_topology = rail)

	# Define the connections between station
	connection_genova_pavia = Rail_connection(station_a = genova_station, 
		station_b = pavia_station, rail_connection_type = Connection_type.HIGH_VELOCITY_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])
	connection_pavia_milano = Rail_connection(station_a = pavia_station, 
		station_b = milano_station, rail_connection_type = Connection_type.HIGH_VELOCITY_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])
	connection_genova_torino = Rail_connection(station_a = pavia_station, 
		station_b = milano_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])
	connection_torino_novara = Rail_connection(station_a = torino_station, 
		station_b = novara_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])
	connection_novara_pavia = Rail_connection(station_a = novara_station, 
		station_b = pavia_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])
	connection_torino_pavia = Rail_connection(station_a = torino_station, 
		station_b = pavia_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	# Define the lines
	genova_milano = Line(type_line = Connection_type.HIGH_VELOCITY_RAIL, 
		stations = (genova_station, pavia_station, milano_station), stops = (1, 1, 1))
	torino_milano = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (torino_station, novara_station, milano_station), stops = (1, 1, 1))
	genova_torino = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, torino_station), stops = (1, 1))

	stations = []
	stations.append([genova_station.position, 1])
	stations.append([pavia_station.position, 1])
	stations.append([milano_station.position, 1])
	stations.append([torino_station.position, 0.5])
	stations.append([novara_station.position, 0.5])

	stations_objects = [genova_station, pavia_station, milano_station, torino_station, novara_station]

	# Defining the train runs
	# convoy 1
	train_run_0 = Train_run(genova_milano, starting_time = 5, from_depot = True)
	# convoy 2
	train_run_1 = Train_run(genova_milano, starting_time = 7, from_depot = True, inverse_train_direction = True)
	# convoy 3
	train_run_2 = Train_run(torino_milano, starting_time = 12, from_depot = True)
	# convoy 4
	train_run_3 = Train_run(torino_milano, starting_time = 3, from_depot = True, inverse_train_direction = True)
	# convoy 5
	train_run_4 = Train_run(genova_torino, starting_time = 11, from_depot = True)
	# convoy 6
	train_run_5 = Train_run(genova_torino, starting_time = 20, from_depot = True, inverse_train_direction = True)

	# Define the convoysss
	FR1001_convoy = Convoy( Type_of_convoy.HIGH_VELOCITY)
	R1002_convoy = Convoy( Type_of_convoy.INTERCITY)	
	R1003_convoy = Convoy( Type_of_convoy.INTERCITY)
	FR1004_convoy = Convoy( Type_of_convoy.HIGH_VELOCITY)
	R1005_convoy = Convoy( Type_of_convoy.INTERCITY)
	R1006_convoy = Convoy( Type_of_convoy.INTERCITY)

	convoys = [FR1001_convoy, R1002_convoy, R1003_convoy, FR1004_convoy, R1005_convoy, R1006_convoy]

	# Add the train runs to convoys
	FR1001_convoy.add_train_run(train_run_0)

	R1002_convoy.add_train_run(train_run_1)

	R1003_convoy.add_train_run(train_run_2)

	FR1004_convoy.add_train_run(train_run_3)

	R1005_convoy.add_train_run(train_run_4)

	R1006_convoy.add_train_run(train_run_5)

	#Calculate plan to do

	timetable_example = calculate_timetable(convoys, rail)

'''
###############################################################
######################   EXAMPLE 0  #########################
###############################################################
'''

if example_training == 0:

	# Import the examples
	from examples.esempio_prova import rail, railway_example, av_line

	stazione_prova = Station('Prova', position = (3,2), capacity = 1, min_wait_time = 1, additional_wait_percent =1, importance = 1, railway_topology = rail)
	stazione_prova_2 = Station('Prova 2', position = (3, 9), capacity = 1, min_wait_time = 1, additional_wait_percent =1, importance = 1, railway_topology = rail)

	connection_prova = Rail_connection(station_a = stazione_prova, 
		station_b = stazione_prova_2, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_prova = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (stazione_prova, stazione_prova_2), stops = (1, 1))

	stations = []
	stations.append([stazione_prova.position, 0.5])
	stations.append([stazione_prova_2.position, 0.5])

	# Define the train runs
	train_run_0 = Train_run(linea_prova, starting_time = 3, from_depot = True)
	train_run_1 = Train_run(linea_prova, starting_time = 40, from_depot = True, inverse_train_direction = True)

	# Define the convoys
	R1079_convoy = Convoy( Type_of_convoy.INTERCITY)
	R1078_convoy = Convoy( Type_of_convoy.INTERCITY)

	convoys = [R1079_convoy, R1078_convoy]

	# Adding the train runs to the convoys
	R1079_convoy.add_train_run(train_run_0)
	R1078_convoy.add_train_run(train_run_1)

	# Generating the timetable
	# The timetable is composed by (station positions, time at which reach the stations, maximum train velocity)
	timetable_example = calculate_timetable(convoys, rail)

'''
###############################################################
######################   Training 0  #########################
###############################################################
'''

if example_training == 'training0':
	# Import the examples

	genova_station = Station('Genova', position = (6,2), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	recco_station = Station('Recco', position = (6,13), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	chiavari_station = Station('Chiavari', position = (6,23), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)

	connection_genova_recco = Rail_connection(station_a = genova_station, 
		station_b = recco_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_recco_chiavari = Rail_connection(station_a = recco_station, 
		station_b = chiavari_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_genova_levante = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, recco_station, chiavari_station), stops = (1, 1, 1))

	stations = []

	stations.append([genova_station.position, 0.5])
	stations.append([recco_station.position, 0.5])
	stations.append([chiavari_station, 0.5])

	train_run_0 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True)

	R102_convoy = Convoy(Type_of_convoy.INTERCITY)

	convoys = [R102_convoy]

	R102_convoy.add_train_run(train_run_0)

	timetable_example = calculate_timetable(convoys, rail)

	# Interruption
	timetable_example.append([[(6,8), (6,13)],[1, 13], 0.5])
	timetable_example.append([[(5,8), (5,13)],[1, 13], 0.5])


'''
###############################################################
######################   Training 0.1 (FOUR STATIONS)  #########################
###############################################################
'''

if example_training == 'training0.1':
	# Import the examples
	from examples.four_stations import rail, railway_example, av_line

	genova_station = Station('Genova', position = (6,2), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	recco_station = Station('Recco', position = (6,13), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	chiavari_station = Station('Chiavari', position = (6,23), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
   
	spezia_station = Station('Spezia', position = (6,32),capacity=3, min_wait_time=[1,1,1],additional_wait_percent=1,importance=1, railway_topology = rail)


	connection_genova_recco = Rail_connection(station_a = genova_station, 
		station_b = recco_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_recco_chiavari = Rail_connection(station_a = recco_station, 
		station_b = chiavari_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_chiavari_spezia = Rail_connection(station_a = chiavari_station,
		station_b = spezia_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable=[0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_genova_levante = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, recco_station, chiavari_station, spezia_station), stops = (1, 1, 1, 1))

	stations = []

	stations.append([genova_station.position, 0.5])
	stations.append([recco_station.position, 0.5])
	stations.append([chiavari_station, 0.5])

	stations.append([spezia_station, 0.5])	

	train_run_0 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True)

	R102_convoy = Convoy(Type_of_convoy.INTERCITY)

	convoys = [R102_convoy]

	R102_convoy.add_train_run(train_run_0)

	timetable_example = calculate_timetable(convoys, rail)

	# Interruption
	timetable_example.append([[(6,8), (6,11)],[1, 225], 0.5])
	timetable_example.append([[(5,8), (5,11)],[1, 225], 0.5])


'''
###############################################################
######################   Training 1  #########################
###############################################################
'''

if example_training == 'training1':
	# Import the examples
	from examples.ferrovia_luca import rail, railway_example, av_line

	genova_station = Station('Genova', position = (6,2), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	recco_station = Station('Recco', position = (6,13), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	chiavari_station = Station('Chiavari', position = (6,23), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)

	connection_genova_recco = Rail_connection(station_a = genova_station, 
		station_b = recco_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_recco_chiavari = Rail_connection(station_a = recco_station, 
		station_b = chiavari_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_genova_levante = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, recco_station, chiavari_station), stops = (1, 1, 1))

	stations = []

	stations.append([genova_station.position, 0.5])
	stations.append([recco_station.position, 0.5])
	stations.append([chiavari_station, 0.5])

	train_run_0 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True)

	train_run_1 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True, inverse_train_direction = True)

	R102_convoy = Convoy(Type_of_convoy.INTERCITY)
	R103_convoy = Convoy(Type_of_convoy.INTERCITY)

	convoys = [R102_convoy, R103_convoy]

	R102_convoy.add_train_run(train_run_0)

	R103_convoy.add_train_run(train_run_1)

	timetable_example = calculate_timetable(convoys, rail)

	# Interruption
	timetable_example.append([[(6,8), (6,11)],[1, 225], 0.5])
	timetable_example.append([[(5,8), (5,11)],[1, 225], 0.5])


'''
###############################################################
######################   Training 1.1  #########################
###############################################################
'''

if example_training == 'training1.1':
	# Import the examples
	from examples.ferrovia_luca import rail, railway_example, av_line

	genova_station = Station('Genova', position = (6,2), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	recco_station = Station('Recco', position = (6,13), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	chiavari_station = Station('Chiavari', position = (6,23), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)

	connection_genova_recco = Rail_connection(station_a = genova_station, 
		station_b = recco_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_recco_chiavari = Rail_connection(station_a = recco_station, 
		station_b = chiavari_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_genova_levante = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, recco_station, chiavari_station), stops = (1, 1, 1))

	stations = []

	stations.append([genova_station.position, 0.5])
	stations.append([recco_station.position, 0.5])
	stations.append([chiavari_station, 0.5])

	train_run_0 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True)

	train_run_1 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True, inverse_train_direction = True)

	R102_convoy = Convoy(Type_of_convoy.INTERCITY)
	R103_convoy = Convoy(Type_of_convoy.INTERCITY)

	convoys = [R102_convoy, R103_convoy]

	R102_convoy.add_train_run(train_run_0)

	R103_convoy.add_train_run(train_run_1)

	timetable_example = calculate_timetable(convoys, rail)

	# Interruption
	timetable_example.append([[(6,8), (6,11)],[1, 225], 0.5])




	'''
###############################################################
######################      DEBUG     #########################
###############################################################
'''

if example_training == 'debug':
	# Import the examples
	from examples.ferrovia_luca import rail, railway_example, av_line

	genova_station = Station('Genova', position = (6,2), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	recco_station = Station('Recco', position = (6,13), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)
	chiavari_station = Station('Chiavari', position = (6,23), capacity = 3, min_wait_time = [1, 1, 1], additional_wait_percent =1, importance = 1, railway_topology = rail)

	connection_genova_recco = Rail_connection(station_a = genova_station, 
		station_b = recco_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	connection_recco_chiavari = Rail_connection(station_a = recco_station, 
		station_b = chiavari_station, rail_connection_type = Connection_type.NORMAL_RAIL,
		max_speed_usable = [0.9, 0.6, 0.3], additional_runtime_percent = [0.1, 0.1, 0.1])

	linea_genova_levante = Line(type_line = Connection_type.NORMAL_RAIL, 
		stations = (genova_station, recco_station, chiavari_station), stops = (1, 1, 1))

	stations = []

	stations.append([genova_station.position, 0.5])
	stations.append([recco_station.position, 0.5])
	stations.append([chiavari_station, 0.5])

	train_run_0 = Train_run(linea_genova_levante, starting_time = 3, from_depot = True)

	R102_convoy = Convoy(Type_of_convoy.INTERCITY)

	convoys = [R102_convoy]

	R102_convoy.add_train_run(train_run_0)

	timetable_example = calculate_timetable(convoys, rail)

## Agent

In [15]:
import os
import json
import itertools
import warnings
from typing import Tuple, List, Callable, Mapping, Optional, Any
from flatland.envs.timetable_utils import Timetable

import numpy as np
from numpy.random.mtrand import RandomState

from flatland.envs.agent_utils import EnvAgent
from flatland.envs.distance_map import DistanceMap
from flatland.envs.rail_env_shortest_paths import get_shortest_paths

def len_handle_none(v):
    if v is not None:
        return len(v)
    else:
        return 0

from flatland.envs.rail_env import RailEnvActions

# Import your own Agent or use RLlib to train agents on Flatland
# As an example we use a random agent instead
class RandomAgent:

	def __init__(self, state_size, action_size):
		self.state_size = state_size
		self.action_size = action_size


	# HERE DEFINE THE ACTIONS TO DO IN CASE THE AGENT IS NOT IN THE DETERMINISTIC PART 
	# For now the agents can only move forward (for DEBUG)
	def act(self, state):
		return RailEnvActions.MOVE_FORWARD

	def step(self, memories):
		"""

		Step function to improve agent by adjusting policy given the observations

		:param memories: SARS Tuple to be
		:return:
		"""

		return

	def save(self, filename):
		# Store the current policy
		return

	def load(self, filename):
		# Load a policy
		return

In [16]:




def timetable_generator(timetable, agents: List[EnvAgent], distance_map: DistanceMap, 
                            agents_hints: dict, np_random: RandomState = None) -> Timetable:
    """
    Calculates earliest departure and latest arrival times for the agents
    This is the new addition in Flatland 3
    Also calculates the max episodes steps based on the density of the timetable

    inputs: 
        agents - List of all the agents rail_env.agents
        distance_map - Distance map of positions to tagets of each agent in each direction
        agent_hints - Uses the number of cities
        np_random - RNG state for seeding
    returns:
        Timetable with the latest_arrivals, earliest_departures and max_episdode_steps
    """
    # max_episode_steps calculation
    if agents_hints:
        city_positions = agents_hints['city_positions']
        num_cities = len(city_positions)
    else:
        num_cities = 2

    timedelay_factor = 4
    alpha = 2
    max_episode_steps = int(timedelay_factor * alpha * \
        (distance_map.rail.width + distance_map.rail.height + (len(agents) / num_cities)))
    
    # Multipliers
    old_max_episode_steps_multiplier = 3.0
    new_max_episode_steps_multiplier = 1.5
    travel_buffer_multiplier = 1.3 # must be strictly lesser than new_max_episode_steps_multiplier
    assert new_max_episode_steps_multiplier > travel_buffer_multiplier
    end_buffer_multiplier = 0.05
    mean_shortest_path_multiplier = 0.2
    
    shortest_paths = get_shortest_paths(distance_map)
    shortest_paths_lengths = [len_handle_none(v) for k,v in shortest_paths.items()]

    # Find mean_shortest_path_time
    agent_speeds = [agent.speed_counter.speed for agent in agents]
    agent_shortest_path_times = np.array(shortest_paths_lengths)/ np.array(agent_speeds)
    mean_shortest_path_time = np.mean(agent_shortest_path_times)

    # Deciding on a suitable max_episode_steps
    longest_speed_normalized_time = np.max(agent_shortest_path_times)
    mean_path_delay = mean_shortest_path_time * mean_shortest_path_multiplier
    max_episode_steps_new = int(np.ceil(longest_speed_normalized_time * new_max_episode_steps_multiplier) + mean_path_delay)
    
    max_episode_steps_old = int(max_episode_steps * old_max_episode_steps_multiplier)

    max_episode_steps = min(max_episode_steps_new, max_episode_steps_old)
    
    end_buffer = int(max_episode_steps * end_buffer_multiplier)
    latest_arrival_max = max_episode_steps-end_buffer

    # Useless unless needed by returning
    earliest_departures = []
    latest_arrivals = []

    for agent in agents:
        i_agent = agent.handle
        agent_shortest_path_time = agent_shortest_path_times[agent.handle]
        agent_travel_time_max = int(np.ceil((agent_shortest_path_time * travel_buffer_multiplier) + mean_path_delay))
        
        departure_window_max = max(latest_arrival_max - agent_travel_time_max, 1)
        
        #earliest_departure = np_random.randint(0, departure_window_max)
        earliest_departure = timetable[i_agent][1][0]
        
        #latest_arrival = earliest_departure + agent_travel_time_max
        latest_arrival = timetable[i_agent][1][-1]
        
        earliest_departures.append(earliest_departure)
        latest_arrivals.append(latest_arrival)

        agent.earliest_departure = earliest_departure
        agent.latest_arrival = latest_arrival

    return Timetable(earliest_departures=earliest_departures, latest_arrivals=latest_arrivals,
                    max_episode_steps=max_episode_steps)

### Rewards and Penalities


Here define the penalities and rewards for the agents

In [17]:
# Penalities 
step_penality = - 0.01               # a step is time passing, so a penality for each step is needed
stop_penality = -0.05                # penalty for stopping a moving agent
reverse_penality = -0.07            # penalty for reversing the march of an agent
skip_penality = 0                   # penalty for skipping a station
target_not_reached_penalty = -20     # penalty for not reaching the final target (depot)
default_skip_penalty = 100
cancellation_factor = 1
cancellation_time_buffer = 0

target_reward = 30         # reward for an agent reaching his final target
station_passage_reward = 10 # reward for an agent reaching intermediate station, the reward is wheighted with the delay of the agent


### Training Flag


In [18]:
example_training = 'training0'

### Environment

In [19]:
"""
Definition of the RailEnv environment.
"""
import random
from random import *

from typing import List, Optional, Dict, Tuple

import numpy as np
from gym.utils import seeding

from flatland.utils.rendertools import RenderTool, AgentRenderVariant
from flatland.core.env import Environment
from flatland.core.env_observation_builder import ObservationBuilder
from flatland.core.grid.grid4 import Grid4Transitions
from flatland.core.transition_map import GridTransitionMap
from flatland.envs.agent_utils import EnvAgent
from flatland.envs.distance_map import DistanceMap

from flatland.envs import malfunction_generators as mal_gen
from flatland.envs import rail_generators as rail_gen
from flatland.envs import line_generators as line_gen
from flatland.envs import persistence
from flatland.envs import agent_chains as ac

from flatland.envs.observations import GlobalObsForRailEnv

from flatland.envs.step_utils.states import TrainState, StateTransitionSignals
from flatland.envs.step_utils.transition_utils import check_valid_action
from flatland.envs.step_utils import action_preprocessing
from flatland.envs.step_utils import env_utils

training = example_training

class RailEnv(Environment):
    """
    RailEnv environment class.

    RailEnv is an environment inspired by a (simplified version of) a rail
    network, in which agents (trains) have to navigate to their target
    locations in the shortest time possible, while at the same time cooperating
    to avoid bottlenecks.

    The valid actions in the environment are:

     -   0: do nothing (continue moving or stay still)
     -   1: turn left at switch and move to the next cell; if the agent was not moving, movement is started
     -   2: move to the next cell in front of the agent; if the agent was not moving, movement is started
     -   3: turn right at switch and move to the next cell; if the agent was not moving, movement is started
     -   4: stop moving
     -   5: invert the direction of march

    Moving forward in a dead-end cell makes the agent turn 180 degrees and step
    to the cell it came from.

    The actions of the agents are executed in order of their handle to prevent
    deadlocks and to allow them to learn relative priorities.

    Reward Function:

    It costs each agent a step_penalty for every time-step taken in the environment. Independent of the movement
    of the agent. Currently all other penalties such as penalty for stopping, starting and invalid actions are set to 0.

    alpha = 1
    beta = 1
    Reward function parameters:

    - invalid_action_penalty = 0
    - step_penalty = -alpha
    - global_reward = beta
    - epsilon = avoid rounding errors
    - stop_penalty = 0  # penalty for stopping a moving agent
    - start_penalty = 0  # penalty for starting a stopped agent

    Stochastic malfunctioning of trains:
    Trains in RailEnv can malfunction if they are halted too often (either by their own choice or because an invalid
    action or cell is selected.

    Every time an agent stops, an agent has a certain probability of malfunctioning. Malfunctions of trains follow a
    poisson process with a certain rate. Not all trains will be affected by malfunctions during episodes to keep
    complexity managable.

    TODO: currently, the parameters that control the stochasticity of the environment are hard-coded in init().
    For Round 2, they will be passed to the constructor as arguments, to allow for more flexibility.

    """
    
    cancellation_factor = 1
    cancellation_time_buffer = 0

    def __init__(self,
                 width,
                 height,
                 max_episode_steps,
                 rail_generator=None,
                 line_generator=None,  # : line_gen.LineGenerator = line_gen.random_line_generator(),
                 number_of_agents=2,
                 obs_builder_object: ObservationBuilder = GlobalObsForRailEnv(),
                 malfunction_generator_and_process_data=None,  # mal_gen.no_malfunction_generator(),
                 malfunction_generator=None,
                 remove_agents_at_target=True,
                 random_seed=None,
                 record_steps=False,
                 close_following=True,
                 ):
        """
        Environment init.

        Parameters
        ----------
        rail_generator : function
            The rail_generator function is a function that takes the width,
            height and agents handles of a  rail environment, along with the number of times
            the env has been reset, and returns a GridTransitionMap object and a list of
            starting positions, targets, and initial orientations for agent handle.
            The rail_generator can pass a distance map in the hints or information for specific line_generators.
            Implementations can be found in flatland/envs/rail_generators.py
        line_generator : function
            The line_generator function is a function that takes the grid, the number of agents and optional hints
            and returns a list of starting positions, targets, initial orientations and speed for all agent handles.
            Implementations can be found in flatland/envs/line_generators.py
        width : int
            The width of the rail map. Potentially in the future,
            a range of widths to sample from.
        height : int
            The height of the rail map. Potentially in the future,
            a range of heights to sample from.
        number_of_agents : int
            Number of agents to spawn on the map. Potentially in the future,
            a range of number of agents to sample from.
        obs_builder_object: ObservationBuilder object
            ObservationBuilder-derived object that takes builds observation
            vectors for each agent.
        remove_agents_at_target : bool
            If remove_agents_at_target is set to true then the agents will be removed by placing to
            RailEnv.DEPOT_POSITION when the agent has reach it's target position.
        random_seed : int or None
            if None, then its ignored, else the random generators are seeded with this number to ensure
            that stochastic operations are replicable across multiple operations
        """
        super().__init__()

        if malfunction_generator_and_process_data is not None:
            print("DEPRECATED - RailEnv arg: malfunction_and_process_data - use malfunction_generator")
            self.malfunction_generator, self.malfunction_process_data = malfunction_generator_and_process_data
        elif malfunction_generator is not None:
            self.malfunction_generator = malfunction_generator
            # malfunction_process_data is not used
            # self.malfunction_generator, self.malfunction_process_data = malfunction_generator_and_process_data
            self.malfunction_process_data = self.malfunction_generator.get_process_data()
        # replace default values here because we can't use default args values because of cyclic imports
        else:
            self.malfunction_generator = mal_gen.NoMalfunctionGen()
            self.malfunction_process_data = self.malfunction_generator.get_process_data()
        
        self.number_of_agents = number_of_agents

        if rail_generator is None:
            rail_generator = rail_gen.sparse_rail_generator()
        self.rail_generator = rail_generator
        if line_generator is None:
            line_generator = line_gen.sparse_line_generator()
        self.line_generator = line_generator

        self.rail: Optional[GridTransitionMap] = None
        self.width = width
        self.height = height

        self.remove_agents_at_target = remove_agents_at_target

        self.obs_builder = obs_builder_object
        self.obs_builder.set_env(self)

        self._max_episode_steps: Optional[int] = max_episode_steps
        self._elapsed_steps = 0

        self.obs_dict = {}
        self.rewards_dict = {}
        self.dev_obs_dict = {}
        self.dev_pred_dict = {}

        self.agents: List[EnvAgent] = []
        self.num_resets = 0
        self.distance_map = DistanceMap(self.agents, self.height, self.width)

        self.action_space = [6]
        
        self.previous_station = [[(-1,0)]] * number_of_agents
        
        self.dones_for_position = [False] * number_of_agents
        

        self._seed()
        if random_seed:
            self._seed(seed=random_seed)

        self.agent_positions = None

        self.run_once = [0]*(self.number_of_agents)   # Flag to check when a train has started   

        # save episode timesteps ie agent positions, orientations.  (not yet actions / observations)
        self.record_steps = record_steps  # whether to save timesteps
        # save timesteps in here: [[[row, col, dir, malfunction],...nAgents], ...nSteps]
        self.cur_episode = []
        self.list_actions = []  # save actions in here

        self.motionCheck = ac.MotionCheck()

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        random.seed(seed)
        self.random_seed = seed

        # Keep track of all the seeds in order
        if not hasattr(self, 'seed_history'):
            self.seed_history = [seed]
        if self.seed_history[-1] != seed:
            self.seed_history.append(seed)

        return [seed]

    def find_indices(self, array, index_to_find):
        indeces = []
        for i in range(len(array)):
            if array[i] == index_to_find:
                indeces.append(i)
        return (indeces)

    # no more agent_handles
    def get_agent_handles(self):
        return range(self.get_num_agents())
    
    def get_num_agents(self) -> int:
        return len(self.agents)

    def add_agent(self, agent):
        """ Add static info for a single agent.
            Returns the index of the new agent.
        """
        self.agents.append(agent)
        return len(self.agents) - 1

    def reset_agents(self):
        """ Reset the agents to their starting positions
        """
        for agent in self.agents:
            agent.reset()
        self.active_agents = [i for i in range(len(self.agents))]

    def action_required(self, agent):
        """
        Check if an agent needs to provide an action

        Parameters
        ----------
        agent: RailEnvAgent
        Agent we want to check

        Returns
        -------
        True: Agent needs to provide an action
        False: Agent cannot provide an action
        """
        return agent.state == TrainState.READY_TO_DEPART or \
               ( agent.state.is_on_map_state() and agent.speed_counter.is_cell_entry )

    def reset(self, regenerate_rail: bool = True, regenerate_schedule: bool = True, *,
              random_seed: int = None) -> Tuple[Dict, Dict]:
        """
        reset(regenerate_rail, regenerate_schedule, activate_agents, random_seed)

        The method resets the rail environment

        Parameters
        ----------
        regenerate_rail : bool, optional
            regenerate the rails
        regenerate_schedule : bool, optional
            regenerate the schedule and the static agents
        random_seed : int, optional
            random seed for environment

        Returns
        -------
        observation_dict: Dict
            Dictionary with an observation for each agent
        info_dict: Dict with agent specific information

        """

        if random_seed:
            self._seed(random_seed)

        optionals = {}
        if regenerate_rail or self.rail is None:

            if "__call__" in dir(self.rail_generator):
                rail, optionals = self.rail_generator(
                    self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)
            elif "generate" in dir(self.rail_generator):
                rail, optionals = self.rail_generator.generate(
                    self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)
            else:
                raise ValueError("Could not invoke __call__ or generate on rail_generator")

            self.rail = rail
            self.height, self.width = self.rail.grid.shape

            # Do a new set_env call on the obs_builder to ensure
            # that obs_builder specific instantiations are made according to the
            # specifications of the current environment : like width, height, etc
            self.obs_builder.set_env(self)

        if optionals and 'distance_map' in optionals:
            self.distance_map.set(optionals['distance_map'])

        if regenerate_schedule or regenerate_rail or self.get_num_agents() == 0:
            agents_hints = None
            if optionals and 'agents_hints' in optionals:
                agents_hints = optionals['agents_hints']

            line = self.line_generator(self.rail, self.number_of_agents, agents_hints, 
                                               self.num_resets, self.np_random)
            self.agents = EnvAgent.from_line(line)

            # Reset distance map - basically initializing
            self.distance_map.reset(self.agents, self.rail)

            # NEW : Time Schedule Generation
            timetable = timetable_generator(timetable_example, self.agents, self.distance_map, 
                                               agents_hints, self.np_random)

            #self._max_episode_steps = 250

            for agent_i, agent in enumerate(self.agents):
                agent.earliest_departure = timetable.earliest_departures[agent_i]         
                agent.latest_arrival = timetable.latest_arrivals[agent_i]
        else:
            self.distance_map.reset(self.agents, self.rail)
        
        # Reset agents to initial states
        self.reset_agents()

        self.run_once = [0]*(self.number_of_agents)   # Flag to check when a train has started

        self.num_resets += 1
        self._elapsed_steps = 0
        
        self.previous_station = [[(-1,0)]] * self.number_of_agents
        
        self.dones_for_position = [False] * self.number_of_agents
        

        # Agent positions map
        self.agent_positions = np.zeros((self.height, self.width), dtype=int) - 1
        self._update_agent_positions_map(ignore_old_positions=False)

        self.dones = dict.fromkeys(list(range(self.get_num_agents())) + ["__all__"], False)

        # Reset the state of the observation builder with the new environment
        self.obs_builder.reset()

        # Empty the episode store of agent positions
        self.cur_episode = []

        info_dict = self.get_info_dict()
        # Return the new observation vectors for each agent
        observation_dict: Dict = self._get_observations()
        if hasattr(self, "renderer") and self.renderer is not None:
            self.renderer = None
        return observation_dict, info_dict


    def _update_agent_positions_map(self, ignore_old_positions=True):
        """ Update the agent_positions array for agents that changed positions """
        for agent in self.agents:
            if not ignore_old_positions or agent.old_position != agent.position:
                if agent.position is not None:
                    self.agent_positions[agent.position] = agent.handle
                if agent.old_position is not None:
                    self.agent_positions[agent.old_position] = -1
    
    def generate_state_transition_signals(self, agent, preprocessed_action, movement_allowed, target_time):
        """ Generate State Transitions Signals used in the state machine """
        st_signals = StateTransitionSignals()
        
        # Malfunction starts when in_malfunction is set to true
        st_signals.in_malfunction = agent.malfunction_handler.in_malfunction

        # Malfunction counter complete - Malfunction ends next timestep
        st_signals.malfunction_counter_complete = agent.malfunction_handler.malfunction_counter_complete

        # Earliest departure reached - Train is allowed to move now
        st_signals.earliest_departure_reached = self._elapsed_steps >= agent.earliest_departure

        # Stop Action Given
        st_signals.stop_action_given = (preprocessed_action == RailEnvActions.STOP_MOVING)

        # Valid Movement action Given
        st_signals.valid_movement_action_given = preprocessed_action.is_moving_action() and movement_allowed

        # Target Reached
        if self._elapsed_steps >= target_time:
            # agent.target = [agent.target[0] - 1, agent.target[1]]
            st_signals.target_reached = env_utils.fast_position_equal(agent.position, agent.target)
        else:
            st_signals.target_reached = False

        # Movement conflict - Multiple trains trying to move into same cell
        # If speed counter is not in cell exit, the train can enter the cell
        st_signals.movement_conflict = (not movement_allowed) and agent.speed_counter.is_cell_exit

        return st_signals

    def _handle_end_reward(self, agent: EnvAgent, timetable) -> int:
        '''
        Handles end-of-episode reward for a particular agent.

        Parameters
        ----------
        agent : EnvAgent
        '''
        i_agent = agent.handle

        if training == 'training0' and i_agent != 0:
            reward = 0
            return reward
        if (training == 'training1' or training == 'training1.1') and i_agent > 1:
            reward = 0
            return reward

        reward = 0

        # Reached intermediated stations?
        #reward = self.intermediate_station_reward(i_agent, timetable)

        # agent done? (arrival_time is not None)
        if agent.state == TrainState.DONE:
            # if agent arrived earlier or on time = 0
            # if agent arrived later = -ve reward based on how late
            reward += target_reward
            self.dones[i_agent] = True
            # DELAY
            delay = min(agent.latest_arrival - agent.arrival_time, 0)
            delay_penalty = delay * 0.7 * 0.7/3 # FORMULA DATA DAL PROF DA AGGIUSTARE
            reward += delay_penalty
            #reward = min(agent.latest_arrival - agent.arrival_time, 0)

        # Agents not done (arrival_time is None)
        else:
            # CANCELLED check (never departed)
            if (agent.state.is_off_map_state()):
                reward += -1 * self.cancellation_factor * \
                    (agent.get_travel_time_on_shortest_path(self.distance_map) + self.cancellation_time_buffer)

            # Departed but never reached
            if (agent.state.is_on_map_state()) and not agent.state == TrainState.MALFUNCTION:
                reward += target_not_reached_penalty
                
                pasted_agent_positions = self.cur_episode
                if pasted_agent_positions == []:
                    reward += default_skip_penalty
                    return reward
                
                stations_to_pass = timetable[i_agent][0]
                
                for positions in range (len(pasted_agent_positions)):
                    # If the agent is passed in at least one station (different from the starting one) no problems
                    if pasted_agent_positions[positions][i_agent] in stations_to_pass[1:]:
                        return reward
                    
                # else, I have to give a skip penalty for all the skipped stations, for now simplified
                # I give an high penalty
                reward += - default_skip_penalty
                return reward
                            
                              
                # DELAY
                #reward = agent.get_current_delay(self._elapsed_steps, self.distance_map)
        
        return reward

    def preprocess_action(self, action, agent):
        """
        Preprocess the provided action
            * Change to DO_NOTHING if illegal action
            * Block all actions when in waiting state
            * Check MOVE_LEFT/MOVE_RIGHT actions on current position else try MOVE_FORWARD
        """
        action = action_preprocessing.preprocess_raw_action(action, agent.state, agent.action_saver.saved_action)
        action = action_preprocessing.preprocess_action_when_waiting(action, agent.state)

        # Try moving actions on current position
        current_position, current_direction = agent.position, agent.direction
        if current_position is None: # Agent not added on map yet
            current_position, current_direction = agent.initial_position, agent.initial_direction
        
        action = action_preprocessing.preprocess_moving_action(action, self.rail, current_position, current_direction)

        # Check transitions, bounts for executing the action in the given position and directon
        if action.is_moving_action() and not check_valid_action(action, self.rail, current_position, current_direction):
            action = RailEnvActions.STOP_MOVING

        return action
    
    def clear_rewards_dict(self):
        """ Reset the rewards dictionary """
        self.rewards_dict = {i_agent: 0 for i_agent in range(len(self.agents))}

    def get_info_dict(self):
        """ 
        Returns dictionary of infos for all agents 
        dict_keys : action_required - 
                    malfunction - Counter value for malfunction > 0 means train is in malfunction
                    speed - Speed of the train
                    state - State from the trains's state machine
        """
        info_dict = {
            'action_required': {i: self.action_required(agent) for i, agent in enumerate(self.agents)},
            'malfunction': {
                i: agent.malfunction_handler.malfunction_down_counter for i, agent in enumerate(self.agents)
            },
            'speed': {i: agent.speed_counter.speed for i, agent in enumerate(self.agents)},
            'state': {i: agent.state for i, agent in enumerate(self.agents)}
        }
        return info_dict
    
    def update_step_rewards(self, i_agent):
        """
        Update the rewards dict for agent id i_agent for every timestep
        """

        """
        action = self.agents[i_agent].action_saver.saved_action
        moving = self.agents[i_agent].moving
        state = self.agents[i_agent].state"""

        reward = 0

        reward = step_penality
        """
        if action == RailEnvActions.REVERSE:
            reward += reverse_penality
        if not moving or state == TrainState.STOPPED:
            reward += stop_penality"""

        self.rewards_dict[i_agent] += reward
        
    def calculate_train_run(self, timetable, i_agent, specific_station_index):
        """[Function to calculate a specific train run for a specific agent]

        Args:
            timetable ([list]): [Timetable]
            i_agent ([int]): [i_agent]
            specific_station_index ([int]): [Index of the station I want to calculate the train run in which is]
        """
        all_train_run = timetable[i_agent][0]
        all_times = timetable[i_agent][1]

        previous_station = all_train_run[specific_station_index]
        
        specific_train_run_stations = []
        specific_train_run_times = []
        specific_train_run = []
        train_run_initial_index = 0
        
        # Starting finding the first station
        if specific_station_index != 0:
            for station_reversed in range(specific_station_index + 1):                    
                if all_train_run[specific_station_index - station_reversed - 1] == previous_station:
                    initial_train_run_station = previous_station
                    if station_reversed == 0:
                        train_run_initial_index = specific_station_index
                        initial_time = all_times[train_run_initial_index]
                    else:
                        train_run_initial_index = specific_station_index - station_reversed
                        initial_time = all_times[train_run_initial_index]
                    break
                elif station_reversed == specific_station_index:
                    initial_train_run_station = previous_station
                    train_run_initial_index = specific_station_index - station_reversed
                    initial_time = all_times[train_run_initial_index]
                    break
                else:
                    previous_station = all_train_run[specific_station_index - station_reversed - 1] 
        # If I'm the first station...
        else:
            initial_train_run_station = previous_station
            initial_time = all_times[0]
        
        previous_station = initial_train_run_station
        previous_time = initial_time   
        
        for stations in range(1 , len(all_train_run)):
            if previous_station == all_train_run[train_run_initial_index + stations]:
                ending_train_run_station = previous_station
                specific_train_run_stations.append(ending_train_run_station)
                specific_train_run_times.append(all_times[train_run_initial_index + stations])
                break
            elif train_run_initial_index + stations == len(all_train_run):
                ending_train_run_station = previous_station
                specific_train_run_stations.append(ending_train_run_station)
                specific_train_run_times.append(all_times[train_run_initial_index + stations])
                break
            else:
                specific_train_run_stations.append(previous_station)
                specific_train_run_times.append(previous_time)
                previous_station = all_train_run[train_run_initial_index + stations]
                previous_time = all_times[train_run_initial_index + stations]
                if train_run_initial_index + stations == len(all_train_run) - 1:
                    specific_train_run_stations.append(previous_station)
                    specific_train_run_times.append(all_times[train_run_initial_index + stations])
                    break
        specific_train_run.append(specific_train_run_stations)
        specific_train_run.append(specific_train_run_times)
                
        return specific_train_run
    
    def calculate_skip_penalty(self, timetable, index_of_station_skipped, station_skipped, time_scheduled,
                               i_agent, station, train_type, specific_train_run):
        
        train_importance = train_type
        station_importance = 0.7
        #station_importance = station.importance   # TODO modificare !!!!
        number_of_station_to_pass = len(specific_train_run)
        
        # array that have to contein all the possible passages from the skipped station for other agents (convoys)
        possible_train_passage = []
        
        for i in range(len(timetable)):
            if i != i_agent:
                num_of_stations = len(timetable[i_agent][0])
                for stations in range(num_of_stations - 1):
                    if timetable[i][0][stations] == station_skipped:        # same station
                        if timetable[i][1][stations] > time_scheduled:      # greater time, so the successive agent
                            if timetable[i][0][stations + 1] == timetable[i_agent][0][index_of_station_skipped + 1]:   # same direction
                                possible_train_passage.append(timetable[i][1][stations] - time_scheduled)
        
        if possible_train_passage == []:
            penalty = - default_skip_penalty
            return penalty
            
        else:
            for i in range(len(possible_train_passage)):
                delay = min(possible_train_passage)
                
            
        penalty = - (delay*train_importance*station_importance)/number_of_station_to_pass
        
        return penalty
    
    def calculate_delay_penalty(self, delay, train_run, station, train_type):
        """[Calculate the penalty of the agent based on the delay, and weighted on the type of train, on the importance of the station and 
        on the number of stations reached by the train]

        Args:
            delay ([int]): [delay of the train in a specific station]
            train_run ([list]): [specific train run i'm doing with my train]
            station ([Station]): [station reached]
            train_type ([convoy.type]): [type of the convoy (regional, intercity, high velocity)]
        """
        number_of_station_to_pass = len(train_run) + 1     
        
        train_importance = train_type
        #station_importance = station.importance
        station_importance = 0.7  # TODO modificalo !!!!!
        
        penalty = - (delay*train_importance*station_importance)/number_of_station_to_pass
        
        return penalty

    def end_of_episode_update(self, have_all_agents_ended, timetable):
        """ 
        Updates made when episode ends
        Parameters: have_all_agents_ended - Indicates if all agents have reached done state
        """
        if have_all_agents_ended or \
           ( (self._max_episode_steps is not None) and (self._elapsed_steps >= self._max_episode_steps)):

            for i_agent, agent in enumerate(self.agents):
                
                reward = self._handle_end_reward(agent, timetable)
                self.rewards_dict[i_agent] += reward
                
                #self.dones[i_agent] = True

            #self.dones["__all__"] = True

    def handle_done_state(self, agent):
        """ Any updates to agent to be made in Done state """
        if agent.state == TrainState.DONE and agent.arrival_time is None:
            agent.arrival_time = self._elapsed_steps
            if self.remove_agents_at_target:
                agent.position = None

    def check_intermediate_station_passage(self, step, i_agent, timetable):
            from operator import itemgetter
            positions = self.cur_episode
            if positions == []:
                return
            reward = 0
            stations_to_pass = timetable[i_agent][0]

            if self.agents[i_agent].position in stations_to_pass and self.agents[i_agent].position not in self.previous_station[i_agent]:

                if self.agents[i_agent].state == TrainState.DONE:
                    self.previous_station[i_agent].append(positions[-1][i_agent])  # Final position reached by the agent
                else:
                    self.previous_station[i_agent].append(positions[step - 1][i_agent])
                
                reward += station_passage_reward

                station_in_which_i_am = self.agents[i_agent].position

                index = self.find_indices(timetable[i_agent][0], station_in_which_i_am)
                difference = []
                for num_station in range(len(index)):
                    difference.append(step - timetable[i_agent][1][index[num_station]])

                index_of_min, value_of_min = min(enumerate(difference), key=itemgetter(1))
                index_of_my_station = index[index_of_min]
                
                station = timetable[i_agent][0][index_of_my_station]
                train_type = timetable[i_agent][2]
                
                specific_train_run = self.calculate_train_run(timetable, i_agent, index_of_my_station)
                
                index_of_my_station_for_my_train_run = specific_train_run[0].index(station)
                
                reward += self.calculate_delay_penalty(value_of_min, specific_train_run, station, train_type)
                # Check if I have skipped a previous station
                if index_of_my_station > 1:
                    previous_stations = specific_train_run[0][0:index_of_my_station_for_my_train_run]
                    flag_station_passed = [False] * (len(previous_stations) - 1)
                    for past_positions in range(len(positions)):
                        for i_previous_station in range(1, index_of_my_station):
                            if positions[past_positions][i_agent] == timetable[i_agent][0][i_previous_station]:
                                self.rewards_dict[i_agent] += 0 
                                flag_station_passed[i_previous_station - 1] = True
                    
                    for i_previous_station in range(len(flag_station_passed)):
                        if not flag_station_passed[i_previous_station]: 
                            station_skipped = previous_stations[0][i_previous_station]
                            time_scheduled_for_station = previous_stations[1][i_previous_station] 
                            reward += self.calculate_skip_penalty(timetable, index_of_my_station, station_skipped, 
                                                             time_scheduled_for_station, i_agent, station, train_type, specific_train_run)
                    self.rewards_dict[i_agent] += reward
                    return
                
                self.rewards_dict[i_agent] += reward


    def step(self, action_dict_: Dict[int, RailEnvActions]):
        """
        Updates rewards for the agents at a step.
        """
        self._elapsed_steps += 1

        # Not allowed to step further once done
        if self.dones["__all__"]:
            raise Exception("Episode is done, cannot call step()")

        self.clear_rewards_dict()

        have_all_agents_ended = True # Boolean flag to check if all agents are done

        self.motionCheck = ac.MotionCheck()  # reset the motion check

        temp_transition_data = {}
        
        for agent in self.agents:
            i_agent = agent.handle

            # Build info dict
            rail, optionals = self.rail_generator(
                self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)

            agent = self.agents[i_agent]

            # Calculate velocities that the agents have to mantein
            velocities = self.check_speed(optionals['agents_hints'])   # TODO variare velocità in base alla stazione da raggiungere     

            agent.speed_counter.speed = velocities[i_agent]

            # Starting time of the agent
            starting_time = optionals['agents_hints']['timetable'][i_agent][1][0]
            ending_time = optionals['agents_hints']['timetable'][i_agent][1][-1]

            agent.earliest_departure = starting_time
            agent.latest_arrival = ending_time

            agent.old_position = agent.position
            agent.old_direction = agent.direction
            # Generate malfunction
            agent.malfunction_handler.generate_malfunction(self.malfunction_generator, self.np_random)

            # Get action for the agent
            action = action_dict_.get(i_agent, RailEnvActions.DO_NOTHING)

            preprocessed_action = self.preprocess_action(action, agent)

            # Save moving actions in not already saved
            agent.action_saver.save_action_if_allowed(preprocessed_action, agent.state)

            # Train's next position can change if current stopped in a fractional speed or train is at cell's exit

            position_update_allowed = agent.speed_counter.is_cell_exit and \
                        not agent.malfunction_handler.malfunction_down_counter > 0 and \
                        not preprocessed_action == RailEnvActions.STOP_MOVING                            

            #position_update_allowed = (agent.speed_counter.is_cell_exit or agent.state == TrainState.STOPPED)

            # Calculate new position
            # Keep agent in same place if already done
            if agent.state == TrainState.DONE:
                new_position, new_direction = agent.position, agent.direction
            elif agent.state == TrainState.MALFUNCTION:
                new_position, new_direction = agent.position, agent.direction
            # Add agent to the map if not on it yet
            elif agent.position is None and agent.action_saver.is_action_saved:
                new_position = agent.initial_position
                new_direction = agent.initial_direction       
            # If movement is allowed apply saved action independent of other agents
            elif agent.action_saver.is_action_saved and position_update_allowed:
                saved_action = agent.action_saver.saved_action
                # Apply action independent of other agents and get temporary new position and direction
                new_position, new_direction  = env_utils.apply_action_independent(saved_action, 
                                                                             self.rail, 
                                                                             agent.position, 
                                                                             agent.direction)
                preprocessed_action = saved_action
            else:
                new_position, new_direction = agent.position, agent.direction

            temp_transition_data[i_agent] = env_utils.AgentTransitionData(position=new_position,
                                                                direction=new_direction,
                                                                preprocessed_action=preprocessed_action)
            
            # This is for storing and later checking for conflicts of agents trying to occupy same cell                                                    
            self.motionCheck.addAgent(i_agent, agent.position, new_position)

        # Find conflicts between trains trying to occupy same cell  TODO controlla i bug
        self.motionCheck.find_conflicts()
        
        for agent in self.agents:
            i_agent = agent.handle

            ## Update positions
            if agent.malfunction_handler.in_malfunction:
                movement_allowed = False
            else:
                # TODO check how the check motion is gestito, fai si che una reverse action sia sempre 
                # possibile ma attenzione quando c'è un treno vicino
                movement_allowed = self.motionCheck.check_motion(i_agent, agent.position) 


            movement_inside_cell = agent.state == TrainState.STOPPED and not agent.speed_counter.is_cell_exit
            movement_allowed = movement_allowed or movement_inside_cell

            # Fetch the saved transition data
            agent_transition_data = temp_transition_data[i_agent]
            preprocessed_action = agent_transition_data.preprocessed_action

            ## Update states
            state_transition_signals = self.generate_state_transition_signals(agent, preprocessed_action, movement_allowed, ending_time)
            agent.state_machine.set_transition_signals(state_transition_signals)
            agent.state_machine.step()

            # Needed when not removing agents at target
            movement_allowed = movement_allowed and agent.state != TrainState.DONE

            # Agent is being added to map
            if agent.state.is_on_map_state():
                if agent.state_machine.previous_state.is_off_map_state():
                    agent.position = agent.initial_position
                    agent.direction = agent.initial_direction
            # Speed counter completes
                elif movement_allowed and (agent.speed_counter.is_cell_exit):
                    agent.position = agent_transition_data.position
                    agent.direction = agent_transition_data.direction
                    agent.state_machine.update_if_reached(agent.position, agent.target)

            # Off map or on map state and position should match
            env_utils.state_position_sync_check(agent.state, agent.position, agent.handle)
                
        self._update_agent_positions_map()
        if self.record_steps:
            self.record_timestep(action_dict_)
            
        for agent in self.agents:
            
            i_agent = agent.handle

            ## Update rewards 
            if agent.state.is_on_map_state():
                if agent.state != TrainState.MALFUNCTION:                           
                    self.update_step_rewards(i_agent)

            # The if condition is important to avoid multiple penalties due to malfunctions occurred in stations
            if agent.state.is_on_map_state() or agent.state == TrainState.DONE:
                if agent.state != TrainState.MALFUNCTION: 
                    self.check_intermediate_station_passage(self._elapsed_steps, i_agent, optionals['agents_hints']['timetable'])
                
            # Handle done state actions, optionally remove agents
            self.handle_done_state(agent)
            
            if training == 'training0':
                if i_agent == 0:
                    have_all_agents_ended &= (agent.state == TrainState.DONE)

            elif training == 'training1' or training == 'training1.1':
                if i_agent < 2:
                    have_all_agents_ended &= (agent.state == TrainState.DONE)

            else:
                have_all_agents_ended &= (agent.state == TrainState.DONE)

            ## Update counters (malfunction and speed)
            agent.speed_counter.update_counter(agent.state, agent.old_position)
                                            #    agent.state_machine.previous_state)
            agent.malfunction_handler.update_counter()

            # Clear old action when starting in new cell
            if agent.speed_counter.is_cell_entry and agent.position is not None:
                agent.action_saver.clear_saved_action()
        
        # Check if episode has ended and update rewards and dones
        self.end_of_episode_update(have_all_agents_ended, optionals['agents_hints']['timetable'])

        return self._get_observations(), self.rewards_dict, self.dones, self.get_info_dict() 
    


    '''def record_timestep(self, dActions):
                    """ 
                    Record the positions and orientations of all agents in memory, in the cur_episode
                    """
                    list_agents_state = []
                    for i_agent in range(self.get_num_agents()):
                        agent = self.agents[i_agent]
                        # the int cast is to avoid numpy types which may cause problems with msgpack
                        # in env v2, agents may have position None, before starting
                        if agent.position is None:
                            pos = (0, 0)
                        else:
                            pos = (int(agent.position[0]), int(agent.position[1]))
                        # print("pos:", pos, type(pos[0]))
                        list_agents_state.append([
                                *pos, int(agent.direction), 
                                agent.malfunction_handler.malfunction_down_counter,  
                                int(agent.state),
                                int(agent.position in self.motionCheck.svDeadlocked)
                                ])
            
                    self.cur_episode.append(list_agents_state)
                    self.list_actions.append(dActions)'''

    def record_timestep(self, dActions):
        """ 
        Record the positions and orientations of all agents in memory, in the cur_episode
        """
        list_agents_state = []
        for i_agent in range(self.get_num_agents()):
            agent = self.agents[i_agent]
            # the int cast is to avoid numpy types which may cause problems with msgpack
            # in env v2, agents may have position None, before starting
            if agent.state == TrainState.DONE and not self.dones_for_position[i_agent]:
                pos = agent.target
                self.dones_for_position[i_agent] = True
                return
                
            elif agent.state.is_off_map_state():
                pos = (-1, 0) 
                
            elif agent.position == None: 
                pos = (-1, 0)
            else:
                pos = (int(agent.position[0]), int(agent.position[1]))
                
            # print("pos:", pos, type(pos[0]))
            list_agents_state.append(pos)

        self.cur_episode.append(list_agents_state)
        self.list_actions.append(dActions)

    def _get_observations(self):
        """
        Utility which returns the dictionary of observations for an agent with respect to environment
        """
        # print(f"_get_obs - num agents: {self.get_num_agents()} {list(range(self.get_num_agents()))}")
        self.obs_dict = self.obs_builder.get_many(list(range(self.get_num_agents())))
        return self.obs_dict

    def get_valid_directions_on_grid(self, row: int, col: int) -> List[int]:
        """
        Returns directions in which the agent can move
        """
        return Grid4Transitions.get_entry_directions(self.rail.get_full_transitions(row, col))

    def _exp_distirbution_synced(self, rate: float) -> float:
        """
        Generates sample from exponential distribution
        We need this to guarantee synchronity between different instances with same seed.
        :param rate:
        :return:
        """
        u = self.np_random.rand()
        x = - np.log(1 - u) * rate
        return x

    def _is_agent_ok(self, agent: EnvAgent) -> bool:
        """
        Check if an agent is ok, meaning it can move and is not malfuncitoinig
        Parameters
        ----------
        agent

        Returns
        -------
        True if agent is ok, False otherwise

        """
        return agent.malfunction_handler.in_malfunction
        

    def save(self, filename):
        print("DEPRECATED call to env.save() - pls call RailEnvPersister.save()")
        persistence.RailEnvPersister.save(self, filename)

    def render(self, mode="rgb_array", gl="PGL", agent_render_variant=AgentRenderVariant.ONE_STEP_BEHIND,
            show_debug=False, clear_debug_text=True, show=False,
            screen_height=600, screen_width=800,
            show_observations=False, show_predictions=False,
            show_rowcols=False, return_image=True):
        """
        This methods provides the option to render the
        environment's behavior as an image or to a window.
        Parameters
        ----------
        mode

        Returns
        -------
        Image if mode is rgb_array, opens a window otherwise
        """
        if not hasattr(self, "renderer") or self.renderer is None:
            self.initialize_renderer(mode=mode, gl=gl,  # gl="TKPILSVG",
                                    agent_render_variant=agent_render_variant,
                                    show_debug=show_debug,
                                    clear_debug_text=clear_debug_text,
                                    show=show,
                                    screen_height=screen_height,  # Adjust these parameters to fit your resolution
                                    screen_width=screen_width)
        return self.update_renderer(mode=mode, show=show, show_observations=show_observations,
                                    show_predictions=show_predictions,
                                    show_rowcols=show_rowcols, return_image=return_image)

    def initialize_renderer(self, mode, gl,
                agent_render_variant,
                show_debug,
                clear_debug_text,
                show,
                screen_height,
                screen_width):
        # Initiate the renderer
        self.renderer = RenderTool(self, gl=gl,  # gl="TKPILSVG",
                                agent_render_variant=agent_render_variant,
                                show_debug=show_debug,
                                clear_debug_text=clear_debug_text,
                                screen_height=screen_height,  # Adjust these parameters to fit your resolution
                                screen_width=screen_width)  # Adjust these parameters to fit your resolution
        self.renderer.show = show
        self.renderer.reset()

    def update_renderer(self, mode, show, show_observations, show_predictions,
                    show_rowcols, return_image):
        """
        This method updates the render.
        Parameters
        ----------
        mode

        Returns
        -------
        Image if mode is rgb_array, None otherwise
        """
        image = self.renderer.render_env(show=show, show_observations=show_observations,
                                show_predictions=show_predictions,
                                show_rowcols=show_rowcols, return_image=return_image)
        if mode == 'rgb_array':
            return image[:, :, :3]

    def close(self):
        """
        This methods closes any renderer window.
        """
        if hasattr(self, "renderer") and self.renderer is not None:
            try:
                if self.renderer.show:
                    self.renderer.close_window()
            except Exception as e:
                print("Could Not close window due to:",e)
            self.renderer = None


    def check_speed(self, agents_hints):

    # Velocity depending on the train type and on the line (Take the minimum between the two possible velocities)
        train_velocities = [0]*self.number_of_agents

        # Check for all the agents
        for i_agent, agent in enumerate(self.agents):

            # the i_agent
            agent = self.agents[i_agent]

            # Check if the agent is in the environment or not
            if agent.position != None:

                # If the agent is in the line i the max velocity is x

                # High velocity line case
                if (agent.position in av_line):  
                    train_velocities[i_agent] = min(1, agents_hints['timetable'][i_agent][2])                
                # Regional line case
                else:
                    train_velocities[i_agent] = min(1/2, agents_hints['timetable'][i_agent][2])

            # If agent is not in the environment deafault velocity is 1/2
            else:
                train_velocities[i_agent] = 1/2

        return train_velocities

# Simulation

Timer

In [20]:
from timeit import default_timer


class Timer(object):
    """
    Utility to measure times.

    TODO:
    - add "lap" method to make it easier to measure average time (+std) when measuring the same thing multiple times.
    """

    def __init__(self):
        self.total_time = 0.0
        self.start_time = 0.0
        self.end_time = 0.0

    def start(self):
        self.start_time = default_timer()

    def end(self):
        self.total_time += default_timer() - self.start_time

    def get(self):
        return self.total_time

    def get_current(self):
        return default_timer() - self.start_time

    def reset(self):
        self.__init__()

    def __repr__(self):
        return self.get()

import numpy as np
from flatland.envs.observations import TreeObsForRailEnv

def max_lt(seq, val):
    """
    Return greatest item in seq for which item < val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    max = 0
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] < val and seq[idx] >= 0 and seq[idx] > max:
            max = seq[idx]
        idx -= 1
    return max


def min_gt(seq, val):
    """
    Return smallest item in seq for which item > val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    min = np.inf
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] >= val and seq[idx] < min:
            min = seq[idx]
        idx -= 1
    return min


def norm_obs_clip(obs, clip_min=-1, clip_max=1, fixed_radius=0, normalize_to_range=False):
    """
    This function returns the difference between min and max value of an observation
    :param obs: Observation that should be normalized
    :param clip_min: min value where observation will be clipped
    :param clip_max: max value where observation will be clipped
    :return: returnes normalized and clipped observatoin
    """
    if fixed_radius > 0:
        max_obs = fixed_radius
    else:
        max_obs = max(1, max_lt(obs, 1000)) + 1

    min_obs = 0  # min(max_obs, min_gt(obs, 0))
    if normalize_to_range:
        min_obs = min_gt(obs, 0)
    if min_obs > max_obs:
        min_obs = max_obs
    if max_obs == min_obs:
        return np.clip(np.array(obs) / max_obs, clip_min, clip_max)
    norm = np.abs(max_obs - min_obs)
    return np.clip((np.array(obs) - min_obs) / norm, clip_min, clip_max)


def _split_node_into_feature_groups(node) -> (np.ndarray, np.ndarray, np.ndarray):
    data = np.zeros(6)
    distance = np.zeros(1)
    agent_data = np.zeros(4)

    data[0] = node.dist_own_target_encountered
    data[1] = node.dist_other_target_encountered
    data[2] = node.dist_other_agent_encountered
    data[3] = node.dist_potential_conflict
    data[4] = node.dist_unusable_switch
    data[5] = node.dist_to_next_branch

    distance[0] = node.dist_min_to_target

    agent_data[0] = node.num_agents_same_direction
    agent_data[1] = node.num_agents_opposite_direction
    agent_data[2] = node.num_agents_malfunctioning
    agent_data[3] = node.speed_min_fractional

    return data, distance, agent_data


def _split_subtree_into_feature_groups(node, current_tree_depth: int, max_tree_depth: int) -> (np.ndarray, np.ndarray, np.ndarray):
    if node == -np.inf:
        remaining_depth = max_tree_depth - current_tree_depth
        # reference: https://stackoverflow.com/questions/515214/total-number-of-nodes-in-a-tree-data-structure
        num_remaining_nodes = int((4 ** (remaining_depth + 1) - 1) / (4 - 1))
        return [-np.inf] * num_remaining_nodes * 6, [-np.inf] * num_remaining_nodes, [-np.inf] * num_remaining_nodes * 4

    data, distance, agent_data = _split_node_into_feature_groups(node)

    if not node.childs:
        return data, distance, agent_data

    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data, sub_distance, sub_agent_data = _split_subtree_into_feature_groups(node.childs[direction], current_tree_depth + 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    return data, distance, agent_data


def split_tree_into_feature_groups(tree, max_tree_depth: int) -> (np.ndarray, np.ndarray, np.ndarray):
    """
    This function splits the tree into three difference arrays of values
    """
    data, distance, agent_data = _split_node_into_feature_groups(tree)

    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data, sub_distance, sub_agent_data = _split_subtree_into_feature_groups(tree.childs[direction], 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    return data, distance, agent_data


def normalize_observation(observation, tree_depth: int, observation_radius=0):
    """
    This function normalizes the observation used by the RL algorithm
    """
    data, distance, agent_data = split_tree_into_feature_groups(observation, tree_depth)

    data = norm_obs_clip(data, fixed_radius=observation_radius)
    distance = norm_obs_clip(distance, normalize_to_range=True)
    agent_data = np.clip(agent_data, -1, 1)
    normalized_obs = np.concatenate((np.concatenate((data, distance)), agent_data))
    return normalized_obs


## RL Utils


In [21]:
# http://pytorch.org/
from os import path
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

!pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

ERROR: torch-0.4.1-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
ERROR: torch-0.4.0-{platform}-linux_x86_64.whl is not a valid wheel filename.
Torch 1.10.0+cu111 CUDA 11.1
Device: cuda:0


In [22]:
import copy
import os
import pickle
import random
from collections import namedtuple, deque, Iterable

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F


class DuelingQNetwork(nn.Module):
    """Dueling Q-network (https://arxiv.org/abs/1511.06581)"""

    def __init__(self, state_size, action_size, hidsize1=128, hidsize2=128):
        super(DuelingQNetwork, self).__init__()

        # value network
        self.fc1_val = nn.Linear(state_size, hidsize1)
        self.fc2_val = nn.Linear(hidsize1, hidsize2)
        self.fc4_val = nn.Linear(hidsize2, 1)

        # advantage network
        self.fc1_adv = nn.Linear(state_size, hidsize1)
        self.fc2_adv = nn.Linear(hidsize1, hidsize2)
        self.fc4_adv = nn.Linear(hidsize2, action_size)

    def forward(self, x):
        val = F.relu(self.fc1_val(x))
        val = F.relu(self.fc2_val(val))
        val = self.fc4_val(val)

        # advantage calculation
        adv = F.relu(self.fc1_adv(x))
        adv = F.relu(self.fc2_adv(adv))
        adv = self.fc4_adv(adv)

        return val + adv - adv.mean()

class Policy:
    def step(self, state, action, reward, next_state, done):
        raise NotImplementedError

    def act(self, state, eps=0.):
        raise NotImplementedError

    def save(self, filename):
        raise NotImplementedError

    def load(self, filename):
        raise NotImplementedError


class DDDQNPolicy(Policy):
    """Dueling Double DQN policy"""

    def __init__(self, state_size, action_size, parameters, evaluation_mode=False):
        self.evaluation_mode = evaluation_mode

        self.state_size = state_size
        self.action_size = action_size
        self.double_dqn = True
        self.hidsize = 1

        if not evaluation_mode:
            self.hidsize = parameters.hidden_size
            self.buffer_size = parameters.buffer_size
            self.batch_size = parameters.batch_size
            self.update_every = parameters.update_every
            self.learning_rate = parameters.learning_rate
            self.tau = parameters.tau
            self.gamma = parameters.gamma
            self.buffer_min_size = parameters.buffer_min_size

        # Device
        if parameters.use_gpu and torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            # print("🐇 Using GPU")
        else:
            self.device = torch.device("cpu")
            # print("🐢 Using CPU")

        # Q-Network
        self.qnetwork_local = DuelingQNetwork(state_size, action_size, hidsize1=self.hidsize, hidsize2=self.hidsize).to(self.device)

        if not evaluation_mode:
            self.qnetwork_target = copy.deepcopy(self.qnetwork_local)
            self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
            self.memory = ReplayBuffer(action_size, self.buffer_size, self.batch_size, self.device)

            self.t_step = 0
            self.loss = 0.0

    def act(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        assert not self.evaluation_mode, "Policy has been initialized for evaluation only."

        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > self.buffer_min_size and len(self.memory) > self.batch_size:
                self._learn()

    def _learn(self):
        experiences = self.memory.sample()
        states, actions, rewards, next_states, dones = experiences

        # Get expected Q values from local model
        q_expected = self.qnetwork_local(states).gather(1, actions)

        if self.double_dqn:
            # Double DQN
            q_best_action = self.qnetwork_local(next_states).max(1)[1]
            q_targets_next = self.qnetwork_target(next_states).gather(1, q_best_action.unsqueeze(-1))
        else:
            # DQN
            q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(-1)

        # Compute Q targets for current states
        q_targets = rewards + (self.gamma * q_targets_next * (1 - dones))

        # Compute loss
        self.loss = F.mse_loss(q_expected, q_targets)

        # Minimize the loss
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        # Update target network
        self._soft_update(self.qnetwork_local, self.qnetwork_target, self.tau)

    def _soft_update(self, local_model, target_model, tau):
        # Soft update model parameters.
        # θ_target = τ*θ_local + (1 - τ)*θ_target
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

    def save(self, filename):
        torch.save(self.qnetwork_local.state_dict(), filename + ".local")
        torch.save(self.qnetwork_target.state_dict(), filename + ".target")

    def load(self, filename):
        if os.path.exists(filename + ".local"):
            self.qnetwork_local.load_state_dict(torch.load(filename + ".local"))
        if os.path.exists(filename + ".target"):
            self.qnetwork_target.load_state_dict(torch.load(filename + ".target"))

    def save_replay_buffer(self, filename):
        memory = self.memory.memory
        with open(filename, 'wb') as f:
            pickle.dump(list(memory)[-500000:], f)

    def load_replay_buffer(self, filename):
        with open(filename, 'rb') as f:
            self.memory.memory = pickle.load(f)

    def test(self):
        self.act(np.array([[0] * self.state_size]))
        self._learn()


Experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, device):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.device = device

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = Experience(np.expand_dims(state, 0), action, reward, np.expand_dims(next_state, 0), done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(self.__v_stack_impr([e.state for e in experiences if e is not None])) \
            .float().to(self.device)
        actions = torch.from_numpy(self.__v_stack_impr([e.action for e in experiences if e is not None])) \
            .long().to(self.device)
        rewards = torch.from_numpy(self.__v_stack_impr([e.reward for e in experiences if e is not None])) \
            .float().to(self.device)
        next_states = torch.from_numpy(self.__v_stack_impr([e.next_state for e in experiences if e is not None])) \
            .float().to(self.device)
        dones = torch.from_numpy(self.__v_stack_impr([e.done for e in experiences if e is not None]).astype(np.uint8)) \
            .float().to(self.device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

    def __v_stack_impr(self, states):
        sub_dim = len(states[0][0]) if isinstance(states[0], Iterable) else 1
        np_states = np.reshape(np.array(states), (len(states), sub_dim))
        return np_states


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  """


## Network Parameters

In [23]:

from argparse import ArgumentParser, Namespace
import sys
sys.argv=['']
del sys


parser = ArgumentParser()
parser.add_argument("-n", "--n_episodes", help="number of episodes to run", default=2500, type=int)
parser.add_argument("-t", "--training_env_config", help="training config id (eg 0 for Test_0)", default=0, type=int)
parser.add_argument("-e", "--evaluation_env_config", help="evaluation config id (eg 0 for Test_0)", default=0, type=int)
parser.add_argument("--n_evaluation_episodes", help="number of evaluation episodes", default=25, type=int)
parser.add_argument("--checkpoint_interval", help="checkpoint interval", default=100, type=int)
parser.add_argument("--eps_start", help="max exploration", default=1.0, type=float)
parser.add_argument("--eps_end", help="min exploration", default=0.01, type=float)
parser.add_argument("--eps_decay", help="exploration decay", default=0.99, type=float)
parser.add_argument("--buffer_size", help="replay buffer size", default=int(1e5), type=int)
parser.add_argument("--buffer_min_size", help="min buffer size to start training", default=0, type=int)
parser.add_argument("--restore_replay_buffer", help="replay buffer to restore", default="", type=str)
parser.add_argument("--save_replay_buffer", help="save replay buffer at each evaluation interval", default=False, type=bool)
parser.add_argument("--batch_size", help="minibatch size", default=128, type=int)
parser.add_argument("--gamma", help="discount factor", default=0.99, type=float)
parser.add_argument("--tau", help="soft update of target parameters", default=1e-3, type=float)
parser.add_argument("--learning_rate", help="learning rate", default=0.5e-4, type=float)
parser.add_argument("--hidden_size", help="hidden size (2 fc layers)", default=128, type=int)
parser.add_argument("--update_every", help="how often to update the network", default=8, type=int)
parser.add_argument("--use_gpu", help="use GPU if available", default=True, type=bool)
parser.add_argument("--num_threads", help="number of threads PyTorch can use", default=1, type=int)
parser.add_argument("--render", help="render 1 episode in 100", default=False, type=bool)
parser.add_argument("--track", help="whether to track using wandb", default=False, type=bool)
training_params = parser.parse_args()


obs_params = {
    "observation_tree_depth": 2,
    "observation_radius": 10,
    "observation_max_path_depth": 30
}

## Modified Files
These files are modified from the real Flatland, so it's important to execute them

In [24]:
import numpy as np
from flatland.envs.step_utils.states import TrainState

class SpeedCounter:
    def __init__(self, speed):
        self._speed = speed
        self.counter = None
        self.reset_counter()

    def update_counter(self, state, old_position):
        # Can't start counting when adding train to the map
        if state == TrainState.MOVING and old_position is not None:
            self.counter += 1
            self.counter = self.counter % (self.max_count + 1)

            

    def __repr__(self):
        return f"speed: {self.speed} \
                 max_count: {self.max_count} \
                 is_cell_entry: {self.is_cell_entry} \
                 is_cell_exit: {self.is_cell_exit} \
                 counter: {self.counter}"

    def reset_counter(self):
        self.counter = 0

    @property
    def is_cell_entry(self):
        return self.counter == 0

    @property
    def is_cell_exit(self):
        return self.counter == self.max_count

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, value):
        self._speed = value

    @property
    def max_count(self):
        return int(1/self._speed) - 1

    def to_dict(self):
        return {"speed": self._speed,
                "counter": self.counter}
    
    def from_dict(self, load_dict):
        self._speed = load_dict['speed']
        self.counter = load_dict['counter']

    def __eq__(self, other):
        return self._speed == other._speed and self.counter == other.counter

class RailEnvActions(IntEnum):
    DO_NOTHING = 0  # implies change of direction in a dead-end!
    MOVE_LEFT = 1
    MOVE_FORWARD = 2
    MOVE_RIGHT = 3
    STOP_MOVING = 4
    REVERSE = 5

    @staticmethod
    def to_char(a: int):
        return {
            0: 'B',
            1: 'L',
            2: 'F',
            3: 'R',
            4: 'S',
            5: 'I',
        }[a]

    @classmethod
    def is_action_valid(cls, action):
        return action in cls._value2member_map_

    def is_moving_action(self):
        return self.value in [self.MOVE_RIGHT, self.MOVE_LEFT, self.MOVE_FORWARD, self.REVERSE]


from flatland.envs.rail_trainrun_data_structures import Waypoint
import numpy as np
import warnings

from typing import Tuple, Optional, NamedTuple, List

from attr import attr, attrs, attrib, Factory

from flatland.core.grid.grid4 import Grid4TransitionsEnum
from flatland.envs.timetable_utils import Line

from flatland.envs.step_utils.action_saver import ActionSaver
from flatland.envs.step_utils.state_machine import TrainStateMachine
from flatland.envs.step_utils.states import TrainState
from flatland.envs.step_utils.malfunction_handler import MalfunctionHandler

Agent = NamedTuple('Agent', [('initial_position', Tuple[int, int]),
                             ('initial_direction', Grid4TransitionsEnum),
                             ('direction', Grid4TransitionsEnum),
                             ('target', Tuple[int, int]),
                             ('moving', bool),
                             ('earliest_departure', int),
                             ('latest_arrival', int),
                             ('handle', int),
                             ('position', Tuple[int, int]),
                             ('arrival_time', int),
                             ('old_direction', Grid4TransitionsEnum),
                             ('old_position', Tuple[int, int]),
                             ('speed_counter', SpeedCounter),
                             ('action_saver', ActionSaver),
                             ('state_machine', TrainStateMachine),
                             ('malfunction_handler', MalfunctionHandler),
                             ])


def load_env_agent(agent_tuple: Agent):
     return EnvAgent(
                        initial_position = agent_tuple.initial_position,
                        initial_direction = agent_tuple.initial_direction,
                        direction = agent_tuple.direction,
                        target = agent_tuple.target,
                        moving = agent_tuple.moving,
                        earliest_departure = agent_tuple.earliest_departure,
                        latest_arrival = agent_tuple.latest_arrival,
                        handle = agent_tuple.handle,
                        position = agent_tuple.position,
                        arrival_time = agent_tuple.arrival_time,
                        old_direction = agent_tuple.old_direction,
                        old_position = agent_tuple.old_position,
                        speed_counter = agent_tuple.speed_counter,
                        action_saver = agent_tuple.action_saver,
                        state_machine = agent_tuple.state_machine,
                        malfunction_handler = agent_tuple.malfunction_handler,
                    )

@attrs
class EnvAgent:
    # INIT FROM HERE IN _from_line()
    initial_position = attrib(type=Tuple[int, int])
    initial_direction = attrib(type=Grid4TransitionsEnum)
    direction = attrib(type=Grid4TransitionsEnum)
    target = attrib(type=Tuple[int, int])
    moving = attrib(default=False, type=bool)

    # NEW : EnvAgent - Schedule properties
    earliest_departure = attrib(default=None, type=int)  # default None during _from_line()
    latest_arrival = attrib(default=None, type=int)  # default None during _from_line()

    handle = attrib(default=None)
    # INIT TILL HERE IN _from_line()

    # Env step facelift
    speed_counter = attrib(default = Factory(lambda: SpeedCounter(1.0)), type=SpeedCounter)
    action_saver = attrib(default = Factory(lambda: ActionSaver()), type=ActionSaver)
    state_machine = attrib(default= Factory(lambda: TrainStateMachine(initial_state=TrainState.WAITING)) , 
                           type=TrainStateMachine)
    malfunction_handler = attrib(default = Factory(lambda: MalfunctionHandler()), type=MalfunctionHandler)

    position = attrib(default=None, type=Optional[Tuple[int, int]])

    # NEW : EnvAgent Reward Handling
    arrival_time = attrib(default=None, type=int)

    # used in rendering
    old_direction = attrib(default=None)
    old_position = attrib(default=None)


    def reset(self):
        """
        Resets the agents to their initial values of the episode. Called after ScheduleTime generation.
        """
        self.position = None
        # TODO: set direction to None: https://gitlab.aicrowd.com/flatland/flatland/issues/280
        self.direction = self.initial_direction
        self.old_position = None
        self.old_direction = None
        self.moving = False

        self.malfunction_handler.reset()

        self.action_saver.clear_saved_action()
        self.speed_counter.reset_counter()
        self.state_machine.reset()

    def to_agent(self) -> Agent:
        return Agent(initial_position=self.initial_position, 
                     initial_direction=self.initial_direction,
                     direction=self.direction,
                     target=self.target,
                     moving=self.moving,
                     earliest_departure=self.earliest_departure, 
                     latest_arrival=self.latest_arrival, 
                     handle=self.handle,
                     position=self.position, 
                     old_direction=self.old_direction, 
                     old_position=self.old_position,
                     speed_counter=self.speed_counter,
                     action_saver=self.action_saver,
                     arrival_time=self.arrival_time,
                     state_machine=self.state_machine,
                     malfunction_handler=self.malfunction_handler)

    def get_shortest_path(self, distance_map) -> List[Waypoint]:
        from flatland.envs.rail_env_shortest_paths import get_shortest_paths # Circular dep fix
        return get_shortest_paths(distance_map=distance_map, agent_handle=self.handle)[self.handle]
        
    def get_travel_time_on_shortest_path(self, distance_map) -> int:
        shortest_path = self.get_shortest_path(distance_map)
        if shortest_path is not None:
            distance = len(shortest_path)
        else:
            distance = 0
        speed = self.speed_counter.speed
        return int(np.ceil(distance / speed))

    def get_time_remaining_until_latest_arrival(self, elapsed_steps: int) -> int:
        return self.latest_arrival - elapsed_steps

    def get_current_delay(self, elapsed_steps: int, distance_map) -> int:
        '''
        +ve if arrival time is projected before latest arrival
        -ve if arrival time is projected after latest arrival
        '''
        return self.get_time_remaining_until_latest_arrival(elapsed_steps) - \
               self.get_travel_time_on_shortest_path(distance_map)


    @classmethod
    def from_line(cls, line: Line):
        """ Create a list of EnvAgent from lists of positions, directions and targets
        """

        #print(line.agent_directions)

        num_agents = len(line.agent_positions)
        
        agent_list = []
        for i_agent in range(num_agents):
            speed = line.agent_speeds[i_agent] if line.agent_speeds is not None else 1.0
            
            agent = EnvAgent(initial_position = line.agent_positions[i_agent],
                            initial_direction = line.agent_directions[i_agent],
                            direction = line.agent_directions[i_agent],
                            target = line.agent_targets[i_agent], 
                            moving = False, 
                            earliest_departure = None,
                            latest_arrival = None,
                            handle = i_agent,
                            speed_counter = SpeedCounter(speed=speed))
            agent_list.append(agent)

        return agent_list

    @classmethod
    def load_legacy_static_agent(cls, static_agents_data: Tuple):
        agents = []
        for i, static_agent in enumerate(static_agents_data):
            if len(static_agent) >= 6:
                agent = EnvAgent(initial_position=static_agent[0], initial_direction=static_agent[1],
                                direction=static_agent[1], target=static_agent[2], moving=static_agent[3],
                                speed_counter=SpeedCounter(static_agent[4]['speed']), handle=i)
            else:
                agent = EnvAgent(initial_position=static_agent[0], initial_direction=static_agent[1],
                                direction=static_agent[1], target=static_agent[2], 
                                moving=False,
                                speed_counter=SpeedCounter(1.0),
                                handle=i)
            agents.append(agent)
        return agents
    
    def __str__(self):
        return f"\n \
                 handle(agent index): {self.handle} \n \
                 initial_position: {self.initial_position}  \n \
                 initial_direction: {self.initial_direction} \n \
                 position: {self.position}  \n \
                 direction: {self.direction}  \n \
                 target: {self.target} \n \
                 old_position: {self.old_position} \n \
                 old_direction {self.old_direction} \n \
                 earliest_departure: {self.earliest_departure}  \n \
                 latest_arrival: {self.latest_arrival} \n \
                 state: {str(self.state)} \n \
                 malfunction_handler: {self.malfunction_handler} \n \
                 action_saver: {self.action_saver} \n \
                 speed_counter: {self.speed_counter}"

    @property
    def state(self):
        return self.state_machine.state

    @state.setter
    def state(self, state):
        self._set_state(state)
    
    def _set_state(self, state):
        warnings.warn("Not recommended to set the state with this function unless completely required")
        self.state_machine.set_state(state)

    @property
    def malfunction_data(self):
        raise ValueError("agent.malunction_data is deprecated, please use agent.malfunction_hander instead")

    @property
    def speed_data(self):
        raise ValueError("agent.speed_data is deprecated, please use agent.speed_counter instead")


"""
TransitionMap and derived classes.
"""

import numpy as np
from importlib_resources import path
from numpy import array

from flatland.core.grid.grid4 import Grid4Transitions
from flatland.core.grid.grid4_utils import get_new_position, get_direction
from flatland.core.grid.grid_utils import IntVector2DArray, IntVector2D
from flatland.core.grid.grid_utils import Vec2dOperations as Vec2d
from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transitions import Transitions
from flatland.utils.ordered_set import OrderedSet
from flatland.core.grid.grid4 import Grid4TransitionsEnum


# TODO are these general classes or for grid4 only?
class TransitionMap:
    """
    Base TransitionMap class.

    Generic class that implements a collection of transitions over a set of
    cells.
    """

    def get_transitions(self, cell_id):
        """
        Return a tuple of transitions available in a cell specified by
        `cell_id` (e.g., a tuple of size of the maximum number of transitions,
        with values 0 or 1, or potentially in between,
        for stochastic transitions).

        Parameters
        ----------
        cell_id : [cell identifier]
            The cell_id object depends on the specific implementation.
            It generally is an int (e.g., an index) or a tuple of indices.

        Returns
        -------
        tuple
            List of the validity of transitions in the cell.

        """
        raise NotImplementedError()

    def set_transitions(self, cell_id, new_transitions):
        """
        Replaces the available transitions in cell `cell_id` with the tuple
        `new_transitions'. `new_transitions` must have
        one element for each possible transition.

        Parameters
        ----------
        cell_id : [cell identifier]
            The cell_id object depends on the specific implementation.
            It generally is an int (e.g., an index) or a tuple of indices.
        new_transitions : tuple
            Tuple of new transitions validitiy for the cell.

        """
        raise NotImplementedError()

    def get_transition(self, cell_id, transition_index):
        """
        Return the status of whether an agent in cell `cell_id` can perform a
        movement along transition `transition_index` (e.g., the NESW direction
        of movement, for agents on a grid).

        Parameters
        ----------
        cell_id : [cell identifier]
            The cell_id object depends on the specific implementation.
            It generally is an int (e.g., an index) or a tuple of indices.
        transition_index : int
            Index of the transition to probe, as index in the tuple returned by
            get_transitions(). e.g., the NESW direction of movement, for agents
            on a grid.

        Returns
        -------
        int or float (depending on Transitions used)
            Validity of the requested transition (e.g.,
            0/1 allowed/not allowed, a probability in [0,1], etc...)

        """
        raise NotImplementedError()

    def set_transition(self, cell_id, transition_index, new_transition):
        """
        Replaces the validity of transition to `transition_index` in cell
        `cell_id' with the new `new_transition`.


        Parameters
        ----------
        cell_id : [cell identifier]
            The cell_id object depends on the specific implementation.
            It generally is an int (e.g., an index) or a tuple of indices.
        transition_index : int
            Index of the transition to probe, as index in the tuple returned by
            get_transitions(). e.g., the NESW direction of movement, for agents
            on a grid.
        new_transition : int or float (depending on Transitions used)
            Validity of the requested transition (e.g.,
            0/1 allowed/not allowed, a probability in [0,1], etc...)

        """
        raise NotImplementedError()


class GridTransitionMap(TransitionMap):
    """
    Implements a TransitionMap over a 2D grid.

    GridTransitionMap implements utility functions.
    """

    def __init__(self, width, height, transitions: Transitions = Grid4Transitions([]), random_seed=None):
        """
        Builder for GridTransitionMap object.

        Parameters
        ----------
        width : int
            Width of the grid.
        height : int
            Height of the grid.
        transitions : Transitions object
            The Transitions object to use to encode/decode transitions over the
            grid.

        """

        self.width = width
        self.height = height
        self.transitions = transitions
        self.random_generator = np.random.RandomState()
        if random_seed is None:
            self.random_generator.seed(12)
        else:
            self.random_generator.seed(random_seed)
        self.grid = np.zeros((height, width), dtype=self.transitions.get_type())

    def get_full_transitions(self, row, column):
        """
        Returns the full transitions for the cell at (row, column) in the format transition_map's transitions.

        Parameters
        ----------
        row: int
        column: int
            (row,column) specifies the cell in this transition map.

        Returns
        -------
        self.transitions.get_type()
            The cell content int the format of this map's Transitions.

        """
        return self.grid[row][column]

    def get_transitions(self, row, column, orientation):
        """
        Return a tuple of transitions available in a cell specified by
        `cell_id` (e.g., a tuple of size of the maximum number of transitions,
        with values 0 or 1, or potentially in between,
        for stochastic transitions).

        Parameters
        ----------
        cell_id : tuple
            The cell_id indices a cell as (column, row, orientation),
            where orientation is the direction an agent is facing within a cell.
            Alternatively, it can be accessed as (column, row) to return the
            full cell content.

        Returns
        -------
        tuple
            List of the validity of transitions in the cell as given by the maps transitions.

        """
        return self.transitions.get_transitions(self.grid[row][column], orientation)

    def set_transitions(self, cell_id, new_transitions):
        """
        Replaces the available transitions in cell `cell_id` with the tuple
        `new_transitions'. `new_transitions` must have
        one element for each possible transition.

        Parameters
        ----------
        cell_id : tuple
            The cell_id indices a cell as (column, row, orientation),
            where orientation is the direction an agent is facing within a cell.
            Alternatively, it can be accessed as (column, row) to replace the
            full cell content.
        new_transitions : tuple
            Tuple of new transitions validitiy for the cell.

        """
        assert len(cell_id) in (2, 3), \
            'GridTransitionMap.set_transitions() ERROR: cell_id tuple must have length 2 or 3.'
        if len(cell_id) == 3:
            self.grid[cell_id[0]][cell_id[1]] = self.transitions.set_transitions(self.grid[cell_id[0]][cell_id[1]],
                                                                                 cell_id[2],
                                                                                 new_transitions)
        elif len(cell_id) == 2:
            self.grid[cell_id[0]][cell_id[1]] = new_transitions

    def get_transition(self, cell_id, transition_index):
        """
        Return the status of whether an agent in cell `cell_id` can perform a
        movement along transition `transition_index` (e.g., the NESW direction
        of movement, for agents on a grid).

        Parameters
        ----------
        cell_id : tuple
            The cell_id indices a cell as (column, row, orientation),
            where orientation is the direction an agent is facing within a cell.
        transition_index : int
            Index of the transition to probe, as index in the tuple returned by
            get_transitions(). e.g., the NESW direction of movement, for agents
            on a grid.

        Returns
        -------
        int or float (depending on Transitions used in the )
            Validity of the requested transition (e.g.,
            0/1 allowed/not allowed, a probability in [0,1], etc...)

        """

        assert len(cell_id) == 3, \
            'GridTransitionMap.get_transition() ERROR: cell_id tuple must have length 2 or 3.'
        return self.transitions.get_transition(self.grid[cell_id[0]][cell_id[1]], cell_id[2], transition_index)

    def set_transition(self, cell_id, transition_index, new_transition, remove_deadends=False):
        """
        Replaces the validity of transition to `transition_index` in cell
        `cell_id' with the new `new_transition`.


        Parameters
        ----------
        cell_id : tuple
            The cell_id indices a cell as (column, row, orientation),
            where orientation is the direction an agent is facing within a cell.
        transition_index : int
            Index of the transition to probe, as index in the tuple returned by
            get_transitions(). e.g., the NESW direction of movement, for agents
            on a grid.
        new_transition : int or float (depending on Transitions used in the map.)
            Validity of the requested transition (e.g.,
            0/1 allowed/not allowed, a probability in [0,1], etc...)

        """
        assert len(cell_id) == 3, \
            'GridTransitionMap.set_transition() ERROR: cell_id tuple must have length 3.'
        self.grid[cell_id[0]][cell_id[1]] = self.transitions.set_transition(
            self.grid[cell_id[0]][cell_id[1]],
            cell_id[2],
            transition_index,
            new_transition,
            remove_deadends)

    def save_transition_map(self, filename):
        """
        Save the transitions grid as `filename`, in npy format.

        Parameters
        ----------
        filename : string
            Name of the file to which to save the transitions grid.

        """
        np.save(filename, self.grid)

    def load_transition_map(self, package, resource):
        """
        Load the transitions grid from `filename` (npy format).
        The load function only updates the transitions grid, and possibly width and height, but the object has to be
        initialized with the correct `transitions` object anyway.

        Parameters
        ----------
        package : string
            Name of the package from which to load the transitions grid.
        resource : string
            Name of the file from which to load the transitions grid within the package.
        override_gridsize : bool
            If override_gridsize=True, the width and height of the GridTransitionMap object are replaced with the size
            of the map loaded from `filename`. If override_gridsize=False, the transitions grid is either cropped (if
            the grid size is larger than (height,width) ) or padded with zeros (if the grid size is smaller than
            (height,width) )

        """
        with path(package, resource) as file_in:
            new_grid = np.load(file_in)

        new_height = new_grid.shape[0]
        new_width = new_grid.shape[1]

        self.width = new_width
        self.height = new_height
        self.grid = new_grid

    def is_dead_end(self, rcPos: IntVector2DArray):
        """
        Check if the cell is a dead-end.

        Parameters
        ----------
        rcPos: Tuple[int,int]
            tuple(row, column) with grid coordinate
        Returns
        -------
        boolean
            True if and only if the cell is a dead-end.
        """
        nbits = 0
        tmp = self.get_full_transitions(rcPos[0], rcPos[1])
        while tmp > 0:
            nbits += (tmp & 1)
            tmp = tmp >> 1
        return nbits == 1

    def is_simple_turn(self, rcPos: IntVector2DArray):
        """
        Check if the cell is a left/right simple turn

        Parameters
        ----------
            rcPos: Tuple[int,int]
                tuple(row, column) with grid coordinate
        Returns
        -------
            boolean
                True if and only if the cell is a left/right simple turn.
        """
        tmp = self.get_full_transitions(rcPos[0], rcPos[1])

        def is_simple_turn(trans):
            all_simple_turns = OrderedSet()
            for trans in [int('0100000000000010', 2),  # Case 1b (8)  - simple turn right
                          int('0001001000000000', 2)  # Case 1c (9)  - simple turn left]:
                          ]:
                for _ in range(3):
                    trans = self.transitions.rotate_transition(trans, rotation=90)
                    all_simple_turns.add(trans)
            return trans in all_simple_turns

        return is_simple_turn(tmp)

    def check_path_exists(self, start: IntVector2DArray, direction: int, end: IntVector2DArray):
        """
        Breath first search for a possible path from one node with a certain orientation to a target node.
        :param start: Start cell rom where we want to check the path
        :param direction: Start direction for the path we are testing
        :param end: Cell that we try to reach from the start cell
        :return: True if a path exists, False otherwise
        """
        visited = OrderedSet()
        stack = [(start, direction)]
        while stack:
            node = stack.pop()
            node_position = node[0]
            node_direction = node[1]

            if Vec2d.is_equal(node_position, end):
                return True
            if node not in visited:
                visited.add(node)

                moves = self.get_transitions(node_position[0], node_position[1], node_direction)
                for move_index in range(4):
                    if moves[move_index]:
                        stack.append((get_new_position(node_position, move_index),
                                      move_index))

        return False

    def cell_neighbours_valid(self, rcPos: IntVector2DArray, check_this_cell=False):
        """
        Check validity of cell at rcPos = tuple(row, column)
        Checks that:
        - surrounding cells have inbound transitions for all the outbound transitions of this cell.

        These are NOT checked - see transition.is_valid:
        - all transitions have the mirror transitions (N->E <=> W->S)
        - Reverse transitions (N -> S) only exist for a dead-end
        - a cell contains either no dead-ends or exactly one

        Returns: True (valid) or False (invalid)
        """
        cell_transition = self.grid[tuple(rcPos)]

        if check_this_cell:
            if not self.transitions.is_valid(cell_transition):
                return False

        gDir2dRC = self.transitions.gDir2dRC  # [[-1,0] = N, [0,1]=E, etc]
        grcPos = array(rcPos)
        grcMax = self.grid.shape

        binTrans = self.get_full_transitions(*rcPos)  # 16bit integer - all trans in/out
        lnBinTrans = array([binTrans >> 8, binTrans & 0xff], dtype=np.uint8)  # 2 x uint8
        g2binTrans = np.unpackbits(lnBinTrans).reshape(4, 4)  # 4x4 x uint8 binary(0,1)
        gDirOut = g2binTrans.any(axis=0)  # outbound directions as boolean array (4)
        giDirOut = np.argwhere(gDirOut)[:, 0]  # valid outbound directions as array of int

        # loop over available outbound directions (indices) for rcPos
        for iDirOut in giDirOut:
            gdRC = gDir2dRC[iDirOut]  # row,col increment
            gPos2 = grcPos + gdRC  # next cell in that direction

            # Check the adjacent cell is within bounds
            # if not, then this transition is invalid!
            if np.any(gPos2 < 0):
                return False
            if np.any(gPos2 >= grcMax):
                return False

            # Get the transitions out of gPos2, using iDirOut as the inbound direction
            # if there are no available transitions, ie (0,0,0,0), then rcPos is invalid
            t4Trans2 = self.get_transitions(*gPos2, iDirOut)
            if any(t4Trans2):
                continue
            else:
                return False
        # If the cell is empty but has incoming connections we return false
        if binTrans < 1:
            connected = 0

            for iDirOut in np.arange(4):
                gdRC = gDir2dRC[iDirOut]  # row,col increment
                gPos2 = grcPos + gdRC  # next cell in that direction

                # Check the adjacent cell is within bounds
                # if not, then ignore it for the count of incoming connections
                if np.any(gPos2 < 0):
                    continue
                if np.any(gPos2 >= grcMax):
                    continue

                # Get the transitions out of gPos2, using iDirOut as the inbound direction
                # if there are no available transitions, ie (0,0,0,0), then rcPos is invalid

                for orientation in range(4):
                    connected += self.get_transition((gPos2[0], gPos2[1], orientation), mirror(iDirOut))
            if connected > 0:
                return False

        return True

    def fix_neighbours(self, rcPos: IntVector2DArray, check_this_cell=False):
        """
        Check validity of cell at rcPos = tuple(row, column)
        Checks that:
        - surrounding cells have inbound transitions for all the outbound transitions of this cell.

        These are NOT checked - see transition.is_valid:
        - all transitions have the mirror transitions (N->E <=> W->S)
        - Reverse transitions (N -> S) only exist for a dead-end
        - a cell contains either no dead-ends or exactly one

        Returns: True (valid) or False (invalid)
        """
        cell_transition = self.grid[tuple(rcPos)]

        if check_this_cell:
            if not self.transitions.is_valid(cell_transition):
                return False

        gDir2dRC = self.transitions.gDir2dRC  # [[-1,0] = N, [0,1]=E, etc]
        grcPos = array(rcPos)
        grcMax = self.grid.shape

        binTrans = self.get_full_transitions(*rcPos)  # 16bit integer - all trans in/out
        lnBinTrans = array([binTrans >> 8, binTrans & 0xff], dtype=np.uint8)  # 2 x uint8
        g2binTrans = np.unpackbits(lnBinTrans).reshape(4, 4)  # 4x4 x uint8 binary(0,1)
        gDirOut = g2binTrans.any(axis=0)  # outbound directions as boolean array (4)
        giDirOut = np.argwhere(gDirOut)[:, 0]  # valid outbound directions as array of int

        # loop over available outbound directions (indices) for rcPos
        for iDirOut in giDirOut:
            gdRC = gDir2dRC[iDirOut]  # row,col increment
            gPos2 = grcPos + gdRC  # next cell in that direction

            # Check the adjacent cell is within bounds
            # if not, then this transition is invalid!
            if np.any(gPos2 < 0):
                return False
            if np.any(gPos2 >= grcMax):
                return False

            # Get the transitions out of gPos2, using iDirOut as the inbound direction
            # if there are no available transitions, ie (0,0,0,0), then rcPos is invalid
            t4Trans2 = self.get_transitions(*gPos2, iDirOut)
            if any(t4Trans2):
                continue
            else:
                self.set_transition((gPos2[0], gPos2[1], iDirOut), mirror(iDirOut), 1)
                return False

        return True

    def fix_transitions(self, rcPos: IntVector2DArray, direction: IntVector2D = -1):
        """
        Fixes broken transitions
        """
        gDir2dRC = self.transitions.gDir2dRC  # [[-1,0] = N, [0,1]=E, etc]
        grcPos = array(rcPos)
        grcMax = self.grid.shape
        # Transition elements
        transitions = RailEnvTransitions()
        cells = transitions.transition_list
        simple_switch_east_south = transitions.rotate_transition(cells[10], 90)
        simple_switch_west_south = transitions.rotate_transition(cells[2], 270)
        symmetrical = cells[6]
        double_slip = cells[5]
        three_way_transitions = [simple_switch_east_south, simple_switch_west_south]
        # loop over available outbound directions (indices) for rcPos

        incoming_connections = np.zeros(4)
        for iDirOut in np.arange(4):
            gdRC = gDir2dRC[iDirOut]  # row,col increment
            gPos2 = grcPos + gdRC  # next cell in that direction

            # Check the adjacent cell is within bounds
            # if not, then ignore it for the count of incoming connections
            if np.any(gPos2 < 0):
                continue
            if np.any(gPos2 >= grcMax):
                continue

            # Get the transitions out of gPos2, using iDirOut as the inbound direction
            # if there are no available transitions, ie (0,0,0,0), then rcPos is invalid
            connected = 0
            for orientation in range(4):
                connected += self.get_transition((gPos2[0], gPos2[1], orientation), mirror(iDirOut))
            if connected > 0:
                incoming_connections[iDirOut] = 1

        number_of_incoming = np.sum(incoming_connections)
        # Only one incoming direction --> Straight line set deadend
        if number_of_incoming == 1:
            if self.get_full_transitions(*rcPos) == 0:
                self.set_transitions(rcPos, 0)
            else:
                self.set_transitions(rcPos, 0)

                for direction in range(4):
                    if incoming_connections[direction] > 0:
                        self.set_transition((rcPos[0], rcPos[1], mirror(direction)), direction, 1)
        # Connect all incoming connections
        if number_of_incoming == 2:
            self.set_transitions(rcPos, 0)

            connect_directions = np.argwhere(incoming_connections > 0)
            self.set_transition((rcPos[0], rcPos[1], mirror(connect_directions[0])), connect_directions[1], 1)
            self.set_transition((rcPos[0], rcPos[1], mirror(connect_directions[1])), connect_directions[0], 1)

        # Find feasible connection for three entries
        if number_of_incoming == 3:
            self.set_transitions(rcPos, 0)
            hole = np.argwhere(incoming_connections < 1)[0][0]
            if direction >= 0:
                switch_type_idx = (direction - hole + 3) % 4
                if switch_type_idx == 0:
                    transition = simple_switch_west_south
                elif switch_type_idx == 2:
                    transition = simple_switch_east_south
                else:
                    transition = self.random_generator.choice(three_way_transitions, 1)
            else:
                transition = self.random_generator.choice(three_way_transitions, 1)
            transition = transitions.rotate_transition(transition, int(hole * 90))
            self.set_transitions((rcPos[0], rcPos[1]), transition)

        # Make a double slip switch
        if number_of_incoming == 4:
            rotation = self.random_generator.randint(2)
            transition = transitions.rotate_transition(double_slip, int(rotation * 90))
            self.set_transitions((rcPos[0], rcPos[1]), transition)
        return True

    def validate_new_transition(self, prev_pos: IntVector2D, current_pos: IntVector2D,
                                new_pos: IntVector2D, end_pos: IntVector2D):
        """
        Utility function to test that a path drawn by a-start algorithm uses valid transition objects.
        We us this to quide a-star as there are many transition elements that are not allowed in RailEnv

        :param prev_pos: The previous position we were checking
        :param current_pos: The current position we are checking
        :param new_pos: Possible child position we move into
        :param end_pos: End cell of path we are drawing
        :return: True if the transition is valid, False if transition element is illegal
        """
        # start by getting direction used to get to current node
        # and direction from current node to possible child node
        new_dir = get_direction(current_pos, new_pos)
        if prev_pos is not None:
            current_dir = get_direction(prev_pos, current_pos)
        else:
            current_dir = new_dir
        # create new transition that would go to child
        new_trans = self.grid[current_pos]
        if prev_pos is None:
            if new_trans == 0:
                # need to flip direction because of how end points are defined
                new_trans = self.transitions.set_transition(new_trans, mirror(current_dir), new_dir, 1)
            else:
                # check if matches existing layout
                new_trans = self.transitions.set_transition(new_trans, current_dir, new_dir, 1)
        else:
            # set the forward path
            new_trans = self.transitions.set_transition(new_trans, current_dir, new_dir, 1)
            # set the backwards path
            new_trans = self.transitions.set_transition(new_trans, mirror(new_dir), mirror(current_dir), 1)
        if Vec2d.is_equal(new_pos, end_pos):
            # need to validate end pos setup as well
            new_trans_e = self.grid[end_pos]
            if new_trans_e == 0:
                # need to flip direction because of how end points are defined
                new_trans_e = self.transitions.set_transition(new_trans_e, new_dir, mirror(new_dir), 1)
            else:
                # check if matches existing layout
                new_trans_e = self.transitions.set_transition(new_trans_e, new_dir, new_dir, 1)

            if not self.transitions.is_valid(new_trans_e):
                return False

        # is transition is valid?
        return self.transitions.is_valid(new_trans)

    # Function to check if a transition is possible/valide or the train/agent can't go there
    def check_transition_is_possible(self, previous_position, current_position, new_position):
        # Finding direction of the agent

        #print(left_rails, right_rails)

        new_dir = get_direction(current_position, new_position)
        if previous_position is not None:
            current_dir = get_direction(previous_position, current_position)
        else:
            current_dir = new_dir

        # checking the possible transitions
        # EXAMPLE OF RESULT OF THE FUNCTION
        # 
        #  [0, 1, 1, 0]   
        #   N  E  S  W
        transitions = self.get_transitions(current_position[0], current_position[1], current_dir)
        #print(previous_position, current_position, transitions,)
        
        # Transition to NORD valid?
        if (new_position[0] == (current_position[0] - 1)) and (transitions[0] == 1):
            if new_dir  == Grid4TransitionsEnum.NORTH:
                if new_position in down_rails:
                    return False
            return True
        # Transition to EAST valid?
        if (new_position[1] == (current_position[1] + 1)) and (transitions[1] == 1):
            if new_dir  == Grid4TransitionsEnum.EAST:
                if new_position in left_rails:
                    return False
            return True
        # Transition to SUD valid?
        if (new_position[0] == (current_position[0] + 1)) and (transitions[2] == 1):
            if new_dir  == Grid4TransitionsEnum.SOUTH:
                if new_position in up_rails:
                    return False
            return True
        # Transition to WEST valid?
        if (new_position[1] == (current_position[1] - 1)) and (transitions[3] == 1):
            if new_dir  == Grid4TransitionsEnum.WEST:
                if new_position in right_rails:
                    return False
            return True
        else:
            return False 

def mirror(dir):
    return (dir + 2) % 4
# TODO: improvement override __getitem__ and __setitem__ (cell contents, not transitions?)




## Simulation import

In [25]:

# In Flatland you can use custom observation builders and predicitors
# Observation builders generate the observation needed by the controller
# Preditctors can be used to do short time prediction which can help in avoiding conflicts in the network
from flatland.envs.malfunction_generators import malfunction_from_params, MalfunctionParameters, ParamMalfunctionGen
from flatland.envs.observations import TreeObsForRailEnv, GlobalObsForRailEnv
# Import the railway generators
from flatland.utils.rendertools import RenderTool, AgentRenderVariant
# Import the agent class
from flatland.envs.step_utils.states import TrainState
from flatland.envs.predictions import ShortestPathPredictorForRailEnv


In [26]:
%%capture
%matplotlib inline

# Helper function to visualize an episode

import matplotlib.pyplot as plt
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"

def display_episode(frames):
    fig, ax = plt.subplots(figsize=(12,12))
    imgplot = plt.imshow(frames[0])
    def animate(i):
        imgplot.set_data(frames[i])
    animation = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames))
    return animation

matplotlib.rcParams['animation.embed_limit'] = 2**128



## Simulation Values

In [27]:
obs_params = {
    "observation_tree_depth": 2,
    "observation_radius": 10,
    "observation_max_path_depth": 30
}

training_flag = example_training

###### TRAINING PARAMETERS #######
n_episodes = 1000
eps_start = 1
eps_end = 0.01
eps_decay = 0.99
max_steps = 250     # 1440 one day
checkpoint_interval = 100
training_id = '0' 
render = False

######### FLAGS ##########
# Flag for the first training
training_flag = example_training
# Flag active in case of interruptions
interruption = True
# Flag to select the agent ----> multi agent or external controller
multi_agent = True
# Flag to save the video or not
video_save = False

## Training 

In [28]:
# Check the maximum possible delay...180 not good for now
def calculate_metric(env, timetable):
    positions = env.cur_episode
    prev_station = 0
    delta = 400 
    metric_result = []
    for i_agent in range(env.get_num_agents()):
        if not env.agents[i_agent].state == TrainState.MALFUNCTION:
            station_vector = [delta] * len(timetable[i_agent][0])
            for i_station in range(len(timetable[i_agent][0])):
                for step in range(len(positions)):
                    if positions[step][i_agent] == timetable[i_agent][0][i_station] and positions[step][i_agent] != prev_station:
                        prev_station = positions[step][i_agent]
                        distance_delay = ((step - timetable[i_agent][1][i_station])**2)**(1/2)
                        station_vector[i_station] = distance_delay
            metric_result.append(station_vector)
    metric_sum = sum(sum(x) for x in metric_result)
    dimension = 0
    for i in range(len(metric_result)):
        for j in range(len(metric_result[i])):
            dimension += 1
    metric_normalized = 1 - (metric_sum / (delta*dimension))
    return metric_normalized

def choose_a_random_training_configuration(env, max_steps):
    case = 0
    if case == 0:
        make_a_deterministic_interruption(env.agents[1], max_steps)
        make_a_deterministic_interruption(env.agents[2], max_steps)
        return    
    elif case == 1:
        env.agents[1].initial_position = (6,15)
        env.agents[2].initial_position = (5,15)
        make_a_deterministic_interruption(env.agents[1], max_steps)
        make_a_deterministic_interruption(env.agents[2], max_steps)
        return
    elif case == 2:
        make_a_deterministic_interruption(env.agents[1], max_steps)
        env.agents[2].malfunction_handler.malfunction_down_counter = max_steps
        return       
    elif case == 3:
        env.agents[1].malfunction_handler.malfunction_down_counter = max_steps
        env.agents[2].initial_position = (5,15)
        make_a_deterministic_interruption(env.agents[2], max_steps)
        return

def format_action_prob(action_probs):
    action_probs = np.round(action_probs, 3)
    actions = ["↻", "←", "↑", "→", "◼", "↓"]

    buffer = ""
    for action, action_prob in zip(actions, action_probs):
        buffer += action + " " + "{:.3f}".format(action_prob) + " "

    return buffer

In [ ]:

# The specs for the custom railway generation are taken from structures.py file
specs = railway_example

widht = len(specs[0])
height = len(specs)

stations_position = []

# Defining the name of the different stations
for i in range(1, len(stations)):
    stations_position.append(stations[i][0])

# Timetable conteins the station where the train should pass, from starting station to aim, and conteins the time at which
# each train has to pass in the station, the last number represent the velocity of train (high velocity, intercity or regional)
# Each row represent a different train

print('------ Calculating the timetable')
print()
timetable = timetable_example

# Number of agents is the rows of the timetable
num_of_agents = len(timetable)

# Check if the timetable is feaseble or not, the function is in schedule_generators
# A timetable is feaseble if the difference of times between two stations is positive and let the trains to reach the successive station
# if two stations are very distant from each other the difference of times can't be very small
seed = 2

# Generating the railway topology, with stations
# Arguments of the generator (specs of the railway, position of stations, timetable)
rail_custom = rail_custom_generator(specs, stations_position, timetable)

transition_map_example, agent_hints = rail_custom(widht, height, num_of_agents)

divide_trains_in_station_rails(timetable, transition_map_example)

control_timetable(timetable,transition_map_example)

for i in range(len(timetable)):
    print(timetable[i])
 
time.sleep(3)

# We can now initiate the schedule generator with the given speed profiles
schedule_generator_custom = custom_schedule_generator(timetable = timetable)

print()
print('------- Calculating the action scheduled')
actions_scheduled = action_to_do(timetable, transition_map_example)

# DEBUG
for i in range(len(actions_scheduled)):
    print()
    print(actions_scheduled[i])
    print()

time.sleep(3)

if multi_agent:

    observation_parameters = Namespace(**obs_params)

    observation_tree_depth = observation_parameters.observation_tree_depth
    observation_radius = observation_parameters.observation_radius
    observation_max_path_depth = observation_parameters.observation_max_path_depth

    # Observation builder
    predictor = ShortestPathPredictorForRailEnv(observation_max_path_depth)
    Observer = TreeObsForRailEnv(max_depth=observation_tree_depth, predictor=predictor)
else:
    Observer = GlobalObsForRailEnv()
    # Ricordarsi che noi vogliamo applicare il RL solo in un intorno della linea dove c'è stata l'interruzione
    # Vogliamo in questo caso un osservatore globale? Forsse meglio valutarne anche uno limitato
    # Ragiona se costruire un osservatore che consideri solo i binari possa essere tanto vantaggioso o no?

stochastic_data = MalfunctionParameters(
    malfunction_rate = 0,  # Rate of malfunction occurence
    min_duration = 15,  # Minimal duration of malfunction
    max_duration = 40  # Max duration of malfunction
)

malfunction_generator = ParamMalfunctionGen(stochastic_data)

env = RailEnv(  width= widht,
                height= height,
                rail_generator = rail_custom,
                line_generator=schedule_generator_custom,
                number_of_agents= num_of_agents,
                malfunction_generator = malfunction_generator,
                obs_builder_object=Observer,
                remove_agents_at_target=True,
                record_steps=True,
                max_episode_steps = max_steps - 1
                )

env.reset()


# If I want I can delay a specific train a specific time
'''
delay_a_train(delay = 250, train = env.agents[1], delay_time = 2, time_of_train_generation = 1, actions = actions_scheduled)
delay_a_train(delay = 250, train = env.agents[2], delay_time = 2, time_of_train_generation = 1, actions = actions_scheduled)
'''

for i in range(len(actions_scheduled)):
    print(actions_scheduled[i])

env_renderer = RenderTool(env,
                          screen_height=300,
                          screen_width=300)  # Adjust these parameters to fit your resolution


# This thing is importand for the RL part, initialize the agent with (state, action) dimension
# Initialize the agent with the parameters corresponding to the environment and observation_builder
if multi_agent:
    n_agents = env.get_num_agents()
    n_features_per_node = env.obs_builder.observation_dim
    n_nodes = sum([np.power(4, i) for i in range(observation_tree_depth + 1)])
    state_size = n_features_per_node * n_nodes

    action_size = env.action_space[0]

    action_count = [0] * action_size
    action_dict = dict()
    agent_obs = [None] * n_agents
    agent_prev_obs = [None] * n_agents
    agent_prev_action = [2] * n_agents
    update_values = [False] * n_agents

    controller = RandomAgent(state_size, action_size)

    # Smoothed values used as target for hyperparameter tuning
    smoothed_normalized_score = -1.0
    smoothed_eval_normalized_score = -1.0
    smoothed_completion = 0.0
    smoothed_eval_completion = 0.0

    train_params = training_params

    policy = DDDQNPolicy(state_size, action_size, train_params)

    # TensorBoard writer
    writer = SummaryWriter()
    writer.add_hparams(vars(train_params), {})
    writer.add_hparams(vars(train_params), {})
    writer.add_hparams(vars(observation_parameters), {})

    training_timer = Timer()
    training_timer.start()

else:
    n_agents = env.get_num_agents()
    state_size = (widht * height)
    # The number of actions is the combination of the number of actions by the number of agents
    action_size = env.action_space[0] ** env.get_num_agents()

    action_count = [0] * action_size
    action_dict = dict()
    agent_obs = [None] * n_agents
    agent_prev_obs = [None] * n_agents
    agent_prev_action = [2] * n_agents
    update_values = [False] * n_agents

    controller = RandomAgent(state_size, action_size)

    q_table = np.zeros([state_size, action_size])


    alpha = 0.1
    gamma = 0.6
    epsilon = 0.1

    # For plotting metrics
    all_epochs = []
    all_penalties = []


# Lets try to enter with all of these agents at the same time
action_dict = dict()

# Now that you have seen these novel concepts that were introduced you will realize that agents don't need to take
# an action at every time step as it will only change the outcome when actions are chosen at cell entry.
# Therefore the environment provides information about what agents need to provide an action in the next step.
# You can access this in the following way.

# Chose an action for each agent
for a in range(env.get_num_agents()):
    action = controller.act(0)
    action_dict.update({a: action})
# Do the environment step

observations, rewards, dones, information = env.step(action_dict)

print("\n The following agents can register an action:")
print("========================================")
for info in information['action_required']:
    print("Agent {} needs to submit an action.".format(info))

# We recommend that you monitor the malfunction data and the action required in order to optimize your training
# and controlling code.

# Let us now look at an episode playing out 

print("\nStart episode...")

# Reset the rendering system
env_renderer.reset()

# Here you can also further enhance the provided observation by means of normalization
# See training navigation example in the baseline repository

score = 0
# Run episode
frame_step = 0

frames = []

os.makedirs("output/frames", exist_ok=True)

for episode_idx in range(n_episodes + 1):
    
    deterministic_interruption_activation = False

    step_timer = Timer()
    reset_timer = Timer()
    learn_timer = Timer()
    preproc_timer = Timer()
    inference_timer = Timer()

    # Reset environment
    reset_timer.start()

    # Reset environment and get initial observations for all agents
    obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
    reset_timer.end()
    for idx in range(env.get_num_agents()):
        tmp_agent = env.agents[idx]
        tmp_agent.speed_counter.speed = 1 / (idx + 1)  # TODO rigestisci le velocità iniziali
    env_renderer.reset()

    if train_params.render:
        env_renderer.set_new_rail()

    score = 0
    nb_steps = 0
    actions_taken = []

    if multi_agent:
        # Build initial agent-specific observations
        for agent in env.get_agent_handles():
            if obs[agent]:
                agent_obs[agent] = normalize_observation(obs[agent], observation_tree_depth, observation_radius=observation_radius)
                agent_prev_obs[agent] = agent_obs[agent].copy()
    else:
        for agent in env.get_agent_handles():
            agent_obs[agent] = obs[agent]
            agent_prev_obs[agent] = agent_obs[agent].copy()

    # Run episode (one day long, 1 step is 1 minute) 1440
    for step in range(max_steps):
        if video_save:
            env_renderer.gl.save_image("output/frames/flatland_frame_step_{:04d}.bmp".format(step))

        inference_timer.start() 

    # Here define the actions to do
        # Broken agents
        # Broken agents
        if training_flag == 'training0' and not deterministic_interruption_activation:
            choose_a_random_training_configuration(env, max_steps)
        if training_flag == 'training1':
            make_a_deterministic_interruption(env.agents[2], max_steps)
            make_a_deterministic_interruption(env.agents[3], max_steps)
        if training_flag == 'training1.1':
            make_a_deterministic_interruption(env.agents[2], max_steps)

        # Chose an action for each agent in the environment
        # If not interruption, the actions to do are stored in a matrix
        #       - each row of the matrix is a train
        #       - each column represent the action the train has to do at each time instant
        
        for a in range(env.get_num_agents()):
            if env.agents[a].state == TrainState.DONE:
                env.dones[a] = True
            if env.agents[a].state == TrainState.MALFUNCTION:
                interruption = True
            if not multi_agent and interruption: # debug 
                break
            if step >= timetable[a][1][0]:
                # Normal plan to follow
                if not interruption and (step - timetable[a][1][0]) < len(actions_scheduled[a]):
                    action = actions_scheduled[a][step - timetable[a][1][0]]
                # Interruption
                elif interruption:
                    if multi_agent:
                        if info['action_required'][a]:
                            update_values[a] = True
                            action = policy.act(agent_obs[a], eps=eps_start)

                            action_count[action] += 1
                            actions_taken.append(action)
                        else:
                            # An action is not required if the train hasn't joined the railway network,
                            # if it already reached its target, or if is currently malfunctioning.
                            update_values[a] = False
                            action = 0
                    else:
                        action = np.random.choice([RailEnvActions.MOVE_FORWARD, RailEnvActions.MOVE_RIGHT, RailEnvActions.MOVE_LEFT, 
                        RailEnvActions.STOP_MOVING, RailEnvActions.REVERSE])
                # choose random from all the possible actions
                else:
                    action = np.random.choice([RailEnvActions.MOVE_FORWARD, RailEnvActions.MOVE_RIGHT, RailEnvActions.MOVE_LEFT, 
                        RailEnvActions.STOP_MOVING, RailEnvActions.REVERSE])

                action_dict.update({a: action})


        inference_timer.end()

        # Environment step which returns the observations for all agents, their corresponding
        # reward and whether their are done
        # Environment step
        step_timer.start()
        next_obs, all_rewards, done, info = env.step(action_dict)
        step_timer.end()

        # Update replay buffer and train agent
        if multi_agent:
            for agent in env.get_agent_handles():
                if update_values[agent] or done['__all__']:
                    # Only learn from timesteps where somethings happened
                    learn_timer.start()
                    policy.step(agent_prev_obs[agent], agent_prev_action[agent], all_rewards[agent], agent_obs[agent], done[agent])
                    learn_timer.end()

                    agent_prev_obs[agent] = agent_obs[agent].copy()
                    agent_prev_action[agent] = action_dict[agent]

                # Preprocess the new observations
                if next_obs[agent]:
                    preproc_timer.start()
                    agent_obs[agent] = normalize_observation(next_obs[agent], observation_tree_depth, observation_radius=observation_radius)
                    preproc_timer.end()

                score += all_rewards[agent]

            nb_steps = step
        else:
            for a in range(env.get_num_agents()):
                controller.step((obs[a], action_dict[a], all_rewards[a], next_obs[a], done[a]))
                score += all_rewards[a]
        obs = next_obs.copy()
        if done['__all__']:
            break
        #break if the first agent has done
        if ((training_flag == 'training0') and (env.dones[0] == True)) or \
            ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
            ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
            break
    print()
    print('Episode Nr. {}\t Score = {}'.format(episode_idx, score))

    # metric most possible near to 0
    metric = calculate_metric(env, timetable)
    
    if multi_agent:
        # Epsilon decay
        eps_start = max(eps_end, eps_decay * eps_start)

        # Collect information about training
        tasks_finished = sum(done[idx] for idx in env.get_agent_handles())
        completion = tasks_finished / max(1, env.get_num_agents())
        normalized_score = score / (max_steps * env.get_num_agents())
        action_probs = action_count / np.sum(action_count)
        action_count = [1] * action_size

        smoothing = 0.99
        smoothed_normalized_score = smoothed_normalized_score * smoothing + normalized_score * (1.0 - smoothing)
        smoothed_completion = smoothed_completion * smoothing + completion * (1.0 - smoothing)

        # Print logs
        if episode_idx % checkpoint_interval == 0:
            '''
            torch.save(policy.qnetwork_local, './checkpoints/' + training_id + '-' + str(episode_idx) + '.pth')
            if save_replay_buffer:
                policy.save_replay_buffer('./replay_buffers/' + training_id + '-' + str(episode_idx) + '.pkl')
            '''

            if train_params.render:
                env_renderer.close_window()

        print(
            '\r🚂 Episode {}'
            '\t 🏆 Score: {:.3f}'
            ' Avg: {:.3f}'
            '\t 💯 Done: {:.2f}%'
            ' Avg: {:.2f}%'
            '\t 🎲 Epsilon: {:.3f} '
            '\t 🔀 Action Probs: {}'
            '\t Metric {}'.format(
                episode_idx,
                normalized_score,
                smoothed_normalized_score,
                100 * completion,
                100 * smoothed_completion,
                eps_start,
                format_action_prob(action_probs),
                metric
            ), end=" ")

    interruption = False
    writer.add_scalar("Reward", score, episode_idx)
    writer.add_scalar("Metric", metric, episode_idx)
    writer.flush()



------ Calculating the timetable

Attention!!! Agent number 1 has a problem in the timetable, times to reach stations 0 and 1 are not right
The time to reach the next station SHOULD BE HIGHER. The minimum time to reach the station should be: 15.0
Attention!!! Agent number 2 has a problem in the timetable, times to reach stations 0 and 1 are not right
The time to reach the next station SHOULD BE HIGHER. The minimum time to reach the station should be: 13.0
[[(6, 2), (6, 13), (6, 23)], [3, 30, 55], 0.5]
[[(6, 8), (5, 13)], [1, 13], 0.5]
[[(5, 8), (5, 13)], [1, 13], 0.5]

------- Calculating the action scheduled

[<RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEnvActions.MOVE_FORWARD: 2>, <RailEn

## Training render

## Reward and metric values 

In [ ]:
%load_ext tensorboard

%tensorboard --logdir=runs

# Tests

## Test 1

In [ ]:
#################
##### TEST 1 ####
#################
# Reset environment and get initial observations for all agents
env.reset()
# Reset the rendering system
env_renderer.reset()
# Change the position of the interrupted agents
env.agents[1].initial_position = (6,10)
env.agents[2].initial_position = (5,10)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    make_a_deterministic_interruption(env.agents[1], max_steps)
    make_a_deterministic_interruption(env.agents[2], max_steps)
    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=False, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break

# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 1 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)


In [ ]:
animation

## Test 2

In [ ]:
#################
##### TEST 2 ####
#################
# Reset the rendering system
env_renderer.reset()
# Reset environment and get initial observations for all agents
obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
# Change the position of the interrupted agents
env.agents[1].initial_position = (6,8)
env.agents[2].initial_position = (-1,0)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    if training_flag == 'training0':
        make_a_deterministic_interruption(env.agents[1], max_steps)
        env.agents[2].malfunction_handler.malfunction_down_counter = max_steps
    if training_flag == 'training1':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        make_a_deterministic_interruption(env.agents[3], max_steps)
    if training_flag == 'training1.1':
        make_a_deterministic_interruption(env.agents[2], max_steps)

    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break

# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 2 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation

## Test 3


In [ ]:
#################
##### TEST 3 ####
#################
# Reset the rendering system
env_renderer.reset()
# Reset environment and get initial observations for all agents
obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
# Change the position of the interrupted agents
env.agents[1].initial_position = (6,14)
env.agents[2].initial_position = (5,14)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    if training_flag == 'training0':
        make_a_deterministic_interruption(env.agents[1], max_steps)
        make_a_deterministic_interruption(env.agents[2], max_steps)
    if training_flag == 'training1':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        make_a_deterministic_interruption(env.agents[3], max_steps)
    if training_flag == 'training1.1':
        make_a_deterministic_interruption(env.agents[2], max_steps)

    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break

# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 3 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")


In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation

## Test 4


In [ ]:
#################
##### TEST 4 ####
#################
# Reset the rendering system
env_renderer.reset()
# Reset environment and get initial observations for all agents
obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
# Change the position of the interrupted agents
env.agents[1].initial_position = (6,14)
env.agents[2].initial_position = (-1,0)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    if training_flag == 'training0':
        make_a_deterministic_interruption(env.agents[1], max_steps)
        env.agents[2].malfunction_handler.malfunction_down_counter = max_steps
    if training_flag == 'training1':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        make_a_deterministic_interruption(env.agents[3], max_steps)
    if training_flag == 'training1.1':
        make_a_deterministic_interruption(env.agents[2], max_steps)

    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break


# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 4 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation

## Test 5

In [ ]:
#################
##### TEST 5 ####
#################
# Reset the rendering system
env_renderer.reset()
# Reset environment and get initial observations for all agents
obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
# Change the position of the interrupted agents
env.agents[2].initial_position = (5,10)
env.agents[1].initial_position = (-1,0)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    if training_flag == 'training0':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        env.agents[1].malfunction_handler.malfunction_down_counter = max_steps
    if training_flag == 'training1':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        make_a_deterministic_interruption(env.agents[3], max_steps)
    if training_flag == 'training1.1':
        make_a_deterministic_interruption(env.agents[2], max_steps)

    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break

# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 5 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation

## Test 6

In [ ]:
#################
##### TEST 6 ####
#################
# Reset the rendering system
env_renderer.reset()
# Reset environment and get initial observations for all agents
obs, info = env.reset(regenerate_rail=True, regenerate_schedule=True)
# Change the position of the interrupted agents
env.agents[2].initial_position = (5,16)
env.agents[1].initial_position = (-1,0)

frame_step = 0
frames = []
score = 0

for step in range(max_steps):
    # Broken agents
    if training_flag == 'training0':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        env.agents[1].malfunction_handler.malfunction_down_counter = max_steps
    if training_flag == 'training1':
        make_a_deterministic_interruption(env.agents[2], max_steps)
        make_a_deterministic_interruption(env.agents[3], max_steps)
    if training_flag == 'training1.1':
        make_a_deterministic_interruption(env.agents[2], max_steps)

    update_values[0] = True
    action = policy.act(agent_obs[0])

    action_count[action] += 1
    actions_taken.append(action)
    action_dict.update({0: action})
    
    next_obs, all_rewards, done, info = env.step(action_dict)
    
    score += all_rewards[0]

    frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    
    if done['__all__']:
        break
    #break if the first agent has done
    if ((training_flag == 'training0') and (env.dones[0] == True)) or \
        ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
        ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
        break

# metric most possible near to 0
metric = calculate_metric(env, timetable)

tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

print(  'Test 6 concluded:'
        '\t 🏆 Score: {:.3f}'
        '\t Agent completed {}'
        '\t Metric {}'.format(
            score,
            tasks_finished,
            metric
        ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation

## Test Prossimamente


In [ ]:
env_renderer = RenderTool(env, gl="PGL", screen_width=500, screen_height=500, show_debug=False)

# Change the position of the interrupted agents
env.reset()
env_renderer.reset()

# Casual Malfunctions for the agents
stochastic_data = MalfunctionParameters(
    malfunction_rate = 0.001,  # Rate of malfunction occurence
    min_duration = 15,  # Minimal duration of malfunction
    max_duration = 60  # Max duration of malfunction
)

malfunction_generator = ParamMalfunctionGen(stochastic_data)

env = RailEnv(  width= widht,
                height= height,
                rail_generator = rail_custom,
                line_generator=schedule_generator_custom,
                number_of_agents= num_of_agents,
                malfunction_generator = malfunction_generator,
                obs_builder_object=Observer,
                remove_agents_at_target=True,
                record_steps=True,
                max_episode_steps = max_steps - 1
                )

env.reset()

#################
##### TEST 3 ####
#################
for episodes in range(20):
  
  frame_step = 0
  frames = []

  for step in range(max_steps):
      # Broken agents
      if training_flag == 'training0':
          make_a_deterministic_interruption(env.agents[1], max_steps)
          make_a_deterministic_interruption(env.agents[2], max_steps)
      if training_flag == 'training1':
          make_a_deterministic_interruption(env.agents[2], max_steps)
          make_a_deterministic_interruption(env.agents[3], max_steps)
      if training_flag == 'training1.1':
          make_a_deterministic_interruption(env.agents[2], max_steps)
      for a in range(env.get_num_agents()):
          update_values[a] = True
          action = policy.act(agent_obs[a], eps=eps_start)

          action_count[action] += 1
          actions_taken.append(action)
          action_dict.update({a: action})
          
          next_obs, all_rewards, done, info = env.step(action_dict)

          frame = env_renderer.render_env(show=False, show_observations=False, show_inactive_agents=True, show_predictions=False, return_image=True)
          frames.append(frame)
          frame_step += 1
      
      if done['__all__']:
          break
      #break if the first agent has done
      if ((training_flag == 'training0') and (env.dones[0] == True)) or \
          ((training_flag == 'training1') and (env.dones[0] == True) and (env.dones[1] == True)) or \
          ((training_flag == 'training1.1') and (env.dones[0] == True) and (env.dones[1] == True)):
          break

  # metric most possible near to 0
  metric = calculate_metric(env, timetable)

  tasks_finished = sum(done[idx] for idx in env.get_agent_handles())

  print()
  print(  'Test 1 concluded:'
          '\t 🏆 Score: {:.3f}'
          '\t Agent completed {}'
          '\t Metric {}'.format(
              score,
              tasks_finished,
              metric
          ), end=" ")

In [ ]:
# RENDER
%%capture
animation = display_episode(frames)

In [ ]:
animation